# Imports

In [ ]:
# Import Essential Modules

"""
Module imports organized according to PEP-8 standards.
Includes error handling for optional Google Cloud dependencies.
"""

# Standard library imports
import asyncio
import functools
import hashlib
import json
import logging
import os
import re
import threading
import time
import unicodedata
import uuid
from dataclasses import dataclass, field
from datetime import datetime, timedelta, timezone
from enum import Enum
from typing import Any, Callable, Dict, List, Optional, Set, Tuple, Union

# Third-party imports (Google Cloud)
try:
    # Google API Core
    from google.api_core import exceptions as gapi_exceptions
    from google.api_core import retry as google_retry

    # Google Auth
    from google.auth import default as get_default_credentials
    from google.auth.exceptions import DefaultCredentialsError

    # Google Cloud Services
    from google.cloud import storage
    from google.cloud import texttospeech_v1 as texttospeech
    from google.cloud.exceptions import (
        Forbidden,
        GoogleCloudError,
        NotFound,
        ServiceUnavailable
    )

    # Vertex AI
    import vertexai

except ImportError as e:
    logging.critical(
        f"Required Google Cloud libraries not installed: {e}. "
        f"Please install: pip install google-cloud-texttospeech "
        f"google-cloud-storage google-cloud-aiplatform"
    )
    raise ImportError(
        "Google Cloud dependencies required. Install with: "
        "pip install google-cloud-texttospeech google-cloud-storage "
        "google-cloud-aiplatform"
    ) from e


# Implementation

# Vertex AI TTS Orchestrator: Cost-Efficient Text-to-Speech on Google Cloud

[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)
[![Python Version](https://img.shields.io/badge/python-3.8+-blue.svg)](https://www.python.org/downloads/)
[![PEP-8 Compliant](https://img.shields.io/badge/code%20style-pep8-orange.svg)](https://www.python.org/dev/peps/pep-0008/)

## Overview

The Vertex AI TTS Orchestrator is a Python-based system designed to provide a highly reliable, scalable, and cost-effective solution for generating high-quality speech from text using Google Cloud's Text-to-Speech API, with deep integration considerations for Vertex AI environments. It intelligently manages API calls, incorporates caching, employs reliability patterns, and offers detailed cost tracking to minimize operational expenses while maximizing performance and audio quality.

This system is engineered for production environments where both audio fidelity and budget adherence are paramount. All components are contained within the `draft_standalone_audio_layer.ipynb` Jupyter Notebook, which includes the core orchestrator logic, helper classes, and a usage example.

## Key Features

*   **Cost-Efficient Synthesis:** Intelligent voice selection (Standard, WaveNet, Neural2) based on configurable cost thresholds and dynamic quality adjustments.
*   **Advanced Caching:** In-memory caching with Time-To-Live (TTL) and Least Recently Used (LRU) eviction policy to reduce redundant API calls and associated costs.
*   **Dynamic Voice Discovery:** Fetches and validates available voices directly from the Google Cloud TTS API at initialization, ensuring up-to-date voice selection.
*   **Robust Reliability Patterns:**
    *   **Circuit Breaker:** Prevents repeated calls to a failing API, allowing services to recover.
    *   **Adaptive Rate Limiter:** Dynamically adjusts request rates to avoid API quotas and adapt to server load.
    *   **Client-Side Retries:** Leverages Google Cloud client library built-in retries for transient network issues.
*   **Asynchronous Operations:** True asynchronous implementation for I/O-bound tasks (TTS synthesis, batch processing) using `asyncio` for enhanced performance and scalability.
*   **Comprehensive Input Validation:** Rigorous validation of all `TTSRequest` parameters to ensure data integrity and prevent erroneous API calls.
*   **Granular Error Handling:** Custom exception hierarchy for precise error identification and handling.
*   **Detailed Metrics & Cost Analysis:** Thread-safe tracking of character usage, costs, cache performance, and API call statistics, with methods to retrieve comprehensive cost analysis reports and optimization recommendations.
*   **SSML Support:** Full support for Speech Synthesis Markup Language (SSML) for fine-grained control over speech output.
*   **Batch Processing:** Concurrent processing of multiple TTS requests with individual error handling and adherence to rate limits and circuit breaker states.
*   **PEP-8 Compliant Code:** Written to the highest Python coding standards for readability and maintainability.
*   **Professional Grade Logging:** Structured and detailed logging throughout the system for observability and debugging.

## Architecture Overview

The orchestrator is built around the `VertexTTSOrchestrator` class, which coordinates several key components:

1.  **`TTSRequest` Dataclass:** A structured and validated representation of a text-to-speech request.
2.  **`VoiceQuality` Enum:** Manages voice quality tiers and associated cost/free-tier information.
3.  **Intelligent Voice Selection:** Dynamically chooses the optimal voice based on request parameters, available voices, and cost-saving strategies.
4.  **Caching Layer:** (`_get_from_cache`, `_put_in_cache`): In-memory cache for storing and retrieving synthesized audio, reducing API calls.
5.  **API Interaction Layer:** Handles communication with the Google Cloud Text-to-Speech API.
    *   **`CircuitBreaker`:** Wraps API calls to manage service unavailability.
    *   **`AdaptiveRateLimiter`:** Controls the rate of API calls.
6.  **`CostMetrics` Dataclass:** Tracks usage and costs in a thread-safe manner.
7.  **Asynchronous Core (`synthesize_speech`, `batch_synthesize`):** Manages concurrent operations efficiently.

## Prerequisites

*   **Python:** Version 3.8 or higher.
*   **Google Cloud Platform (GCP) Account:**
    *   A GCP project with billing enabled.
    *   The **Text-to-Speech API** and **Vertex AI API** must be enabled for your project.
*   **Google Cloud SDK (`gcloud`):** Installed and configured with credentials authorized to access your GCP project (e.g., via `gcloud auth application-default login`). Alternatively, a service account key JSON file can be used.
*   **Jupyter Notebook Environment:** To run `draft_standalone_audio_layer.ipynb`.

## Setup and Installation

1.  **Clone/Download the Notebook:**
    Obtain the `draft_standalone_audio_layer.ipynb` file.

2.  **Install Dependencies:**
    The orchestrator relies on several Google Cloud client libraries and standard Python packages. Install them using pip:
    ```bash
    pip install google-cloud-texttospeech google-cloud-storage google-cloud-aiplatform asyncio logging
    ```
    *(Note: `google-cloud-storage` is listed due to its import, though the GCS caching feature is currently commented out in favor of in-memory caching in the provided code. Other standard libraries like `json`, `hashlib`, etc., are part of Python's standard distribution.)*

3.  **Google Cloud Authentication:**
    Ensure your environment is authenticated to Google Cloud. The recommended methods are:
    *   **Application Default Credentials (ADC):** Run `gcloud auth application-default login` in your terminal.
    *   **Service Account Key:** Download a service account JSON key file with appropriate permissions (e.g., "Text-to-Speech API User", "Vertex AI User"). Set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to the path of this JSON file, or pass the path directly to the `VertexTTSOrchestrator` during initialization.

## Configuration

The orchestrator primarily requires the following configurations, ideally set via environment variables for production deployments:

*   `GOOGLE_CLOUD_PROJECT`: Your Google Cloud Project ID. (Required)
*   `GOOGLE_CLOUD_REGION`: The Google Cloud region for Vertex AI operations (e.g., `us-central1`). Defaults to `us-central1` if not set. (Optional, but recommended)
*   `GOOGLE_APPLICATION_CREDENTIALS`: Path to your service account key JSON file. (Optional if using ADC via `gcloud`)

These are used by the `main_usage_example()` function within the notebook. When instantiating `VertexTTSOrchestrator` directly, these (and other parameters like `cost_threshold_usd`) can be passed as arguments.

## Usage

The `draft_standalone_audio_layer.ipynb` notebook contains all class definitions and the `main_usage_example()` function. To use the orchestrator:

1.  **Open the Notebook:** Launch the `.ipynb` file in a Jupyter Notebook environment (e.g., JupyterLab, VS Code with Python extension).
2.  **Run All Cells (or relevant cells):** Execute the cells containing the class definitions (`TTSOrchestratorError` and its descendants, `VoiceQuality`, `TTSRequest`, `CostMetrics`, `CircuitBreaker`, `AdaptiveRateLimiter`, `VertexTTSOrchestrator`).
3.  **Instantiate and Use the Orchestrator:**

    ```python
    # Ensure necessary imports and class definitions from the notebook are executed first.

    # Example: Initialize the orchestrator (ensure project_id is set)
    # project_id = "your-gcp-project-id" # Or retrieved from os.getenv
    # orchestrator = VertexTTSOrchestrator(project_id=project_id)

    # --- Single TTS Request ---
    # request_data = TTSRequest(
    #     text="Hello, world! This is a test of the TTS orchestrator.",
    #     language_code="en-US",
    #     quality_tier=VoiceQuality.NEURAL2
    # )
    # audio_bytes, metadata = await orchestrator.synthesize_speech(request_data)
    # if audio_bytes:
    #     with open("output_single.mp3", "wb") as f:
    #         f.write(audio_bytes)
    #     print(f"Single synthesis successful. Metadata: {metadata}")
    # else:
    #     print(f"Single synthesis failed. Error: {metadata.get('error')}")

    # --- Batch TTS Requests ---
    # batch_requests = [
    #     TTSRequest(text="First batch item.", language_code="en-US", priority=1),
    #     TTSRequest(text="Second batch item, in French.", language_code="fr-FR", quality_tier=VoiceQuality.STANDARD, priority=2)
    # ]
    # results = await orchestrator.batch_synthesize(batch_requests)
    # for i, (audio_bytes, metadata) in enumerate(results):
    #     if audio_bytes:
    #         with open(f"output_batch_{i}.mp3", "wb") as f:
    #             f.write(audio_bytes)
    #         print(f"Batch item {i} successful. Metadata: {metadata}")
    #     else:
    #         print(f"Batch item {i} failed. Error: {metadata.get('error')}")

    # --- Get Cost Analysis ---
    # cost_report = orchestrator.get_cost_analysis()
    # import json
    # print(json.dumps(cost_report, indent=2))
    ```

    Refer to the `main_usage_example()` function in the notebook for a comprehensive demonstration.

## Key Components

*   **`VertexTTSOrchestrator`:** The main class orchestrating all TTS operations.
*   **`TTSRequest`:** Dataclass for defining and validating individual TTS requests.
*   **`VoiceQuality` (Enum):** Defines available voice quality tiers and their cost implications.
*   **`CostMetrics`:** Thread-safe dataclass for tracking usage, costs, and performance metrics.
*   **`CircuitBreaker`:** Implements the circuit breaker pattern for API call resilience.
*   **`AdaptiveRateLimiter`:** Manages API request rates dynamically.
*   **Custom Exceptions:** A hierarchy of custom exceptions (`TTSOrchestratorError`, `ValidationError`, etc.) for granular error reporting.

## Error Handling

The system employs a robust error handling strategy:
*   **Input Validation:** `TTSRequest` validates all inputs upon instantiation.
*   **Custom Exceptions:** Specific exceptions are raised for different error conditions (e.g., `ConfigurationError`, `SynthesisFailedError`, `APICallError`).
*   **Circuit Breaker & Rate Limiter:** These components inherently handle and mitigate errors related to service unavailability or rate limiting.
*   **Detailed Logging:** Errors are logged with contextual information for easier debugging.

## Cost Optimization Strategies

The orchestrator implements several strategies to minimize TTS costs:
1.  **Intelligent Voice Selection:** Prioritizes standard voices if cost thresholds are approached, even if premium voices are requested.
2.  **Caching:** Stores frequently requested audio, significantly reducing API calls for repeated content.
3.  **Free Tier Awareness:** The `_calculate_estimated_cost` method considers Google Cloud's free tier limits when estimating costs for individual requests.
4.  **Efficient Batching:** Processes multiple requests concurrently, optimizing resource utilization (though cost per character remains the primary driver).

## Logging and Monitoring

*   The system uses Python's standard `logging` module.
*   Logs are structured to include timestamps, logger names, levels, thread names, and messages.
*   The `CostMetrics` class provides data that can be periodically snapshot and sent to monitoring systems for tracking TTS usage, costs, and performance over time.

## Running the Example

The `draft_standalone_audio_layer.ipynb` notebook includes a `main_usage_example()` asynchronous function. To run it:
1.  Ensure all class definitions in the notebook are executed.
2.  Set the `GOOGLE_CLOUD_PROJECT` environment variable (and optionally `GOOGLE_CLOUD_REGION` and `GOOGLE_APPLICATION_CREDENTIALS`).
3.  Execute the cell containing the `if __name__ == "__main__":` block, which calls `asyncio.run(main_usage_example())`.

Output audio files (`output_single_*.mp3`, `output_batch_*.mp3`) will be saved in the same directory as the notebook, and a cost analysis report will be printed to the console and logs.

## Limitations and Future Work

*   **In-Memory Cache:** The current caching mechanism is in-memory. For distributed or persistent caching in larger production systems, integration with external cache stores like Redis or Memcached would be beneficial.
*   **Single Notebook Structure:** While convenient for demonstration, for use as a library in larger projects, the classes should ideally be refactored into separate Python modules (`.py` files) for better organization and importability.
*   **Dynamic Region/Endpoint Configuration:** While the region is configurable, more advanced endpoint management for Vertex AI or TTS could be added if required.
*   **Advanced Analytics:** The current cost analysis is rule-based. More sophisticated trend analysis or predictive cost forecasting could be built on top of the collected `CostMetrics`.

## Contributing

Contributions are welcome to enhance the capabilities and robustness of the Vertex AI TTS Orchestrator. Please follow these guidelines:

1.  **Fork the Repository** (if this were a Git repository).
2.  **Create a Feature Branch:** (`git checkout -b feature/YourFeatureName`)
3.  **Adhere to Coding Standards:**
    *   Ensure all code is compliant with PEP-8.
    *   Write comprehensive unit tests for new features or bug fixes.
    *   Update documentation (docstrings, README) as necessary.
    *   Maintain the high level of in-text commenting established in the project.
4.  **Commit Your Changes:** (`git commit -m 'Add some YourFeatureName'`)
5.  **Push to the Branch:** (`git push origin feature/YourFeatureName`)
6.  **Open a Pull Request.**

## License

This project is licensed under the MIT License - see the [LICENSE.md](LICENSE.md) file (conceptual, as no separate file is provided for a single notebook) for details.

*(The MIT License is a permissive free software license originating at the Massachusetts Institute of Technology (MIT). As a permissive license, it puts only very limited restriction on reuse and has, therefore, high license compatibility.)*

--

This README provides a comprehensive guide to understanding, setting up, and utilizing the Vertex AI TTS Orchestrator. It is designed to meet the rigorous standards of professional, implementation-grade software documentation.

In [ ]:
# Custom Exception Types for better error differentiation
# Defines the base custom exception for all orchestrator-specific errors.
class TTSOrchestratorError(Exception):
    """Base exception for the TTS Orchestrator."""
    # Indicates that this class definition is complete and no further methods are added here.
    pass

# Defines a custom exception for errors occurring during orchestrator initialization.
class InitializationError(TTSOrchestratorError):
    """Error during orchestrator initialization."""
    # Indicates that this class definition is complete and no further methods are added here.
    pass

# Defines a custom exception for errors related to configuration values.
class ConfigurationError(TTSOrchestratorError):
    """Error related to configuration values."""
    # Indicates that this class definition is complete and no further methods are added here.
    pass

# Defines a custom exception, 'ValidationError', inheriting from both 'TTSOrchestratorError' and the built-in 'ValueError',
# specifically for errors related to invalid input data provided to the system.
class ValidationError(TTSOrchestratorError, ValueError):
    """Error for invalid input data."""
    # This 'pass' statement signifies that the class body is intentionally empty,
    # as the class derives its primary functionality and identity from its parent classes and its specific semantic role.
    pass

# Defines a custom exception, 'VoiceSelectionError', inheriting from 'TTSOrchestratorError',
# designated for errors encountered during the intelligent voice selection process.
class VoiceSelectionError(TTSOrchestratorError):
    """Error during intelligent voice selection."""
    # This 'pass' statement signifies that the class body is intentionally empty,
    # as the class derives its primary functionality and identity from its parent class and its specific semantic role.
    pass

# Defines a custom exception, 'SynthesisFailedError', inheriting from 'TTSOrchestratorError',
# specifically for errors that occur during the speech synthesis process itself.
class SynthesisFailedError(TTSOrchestratorError):
    """Error during speech synthesis."""
    # This 'pass' statement signifies that the class body is intentionally empty,
    # as the class derives its primary functionality and identity from its parent class and its specific semantic role.
    pass

# Defines a custom exception, 'CacheError', inheriting from 'TTSOrchestratorError',
# designated for errors specifically related to caching operations within the system.
class CacheError(TTSOrchestratorError):
    """Error related to caching operations."""
    # This 'pass' statement signifies that the class body is intentionally empty,
    # as the class derives its primary functionality and identity from its parent class and its specific semantic role.
    pass

# Defines a custom exception, 'APICallError', inheriting from 'TTSOrchestratorError',
# intended for errors encountered during an API call to an external Google Cloud service.
class APICallError(TTSOrchestratorError):
    """Error during an API call to a Google Cloud service."""
    # This 'pass' statement signifies that the class body is intentionally empty,
    # as the class derives its primary functionality and identity from its parent class and its specific semantic role.
    pass


In [ ]:
# `VoiceQuality(Enum)` (including its methods)**
# Configure root logger early for comprehensive logging
# This line configures the basic settings for the root logger.
logging.basicConfig(
    # Set the minimum logging level to INFO.
    level=logging.INFO,
    # Define the format string for log messages.
    format='%(asctime)s - %(name)s - %(levelname)s - %(threadName)s - %(message)s',
    # Define the date format for the 'asctime' field in log messages.
    datefmt='%Y-%m-%d %H:%M:%S'
)
# Get a logger instance specific to the current module (__name__).
logger = logging.getLogger(__name__)

# Defines an enumeration 'VoiceQuality' inheriting from 'Enum', representing different voice quality tiers
# with associated cost implications and pricing details for Google Cloud Text-to-Speech.
class VoiceQuality(Enum):
    """
    Enumerates voice quality tiers with associated cost implications and pricing details.

    Each enum value represents a different pricing tier in Google Cloud Text-to-Speech.
    Accurate pricing is crucial for cost-efficient operations. As of June 2025 (illustrative):
    - STANDARD: Typically offers a larger free tier (e.g., 4M characters/month), then a lower price (e.g., $4/1M characters).
    - WAVENET: A premium tier, often with a smaller free tier (e.g., 1M characters/month), then a higher price (e.g., $16/1M characters).
    - NEURAL2: Another premium tier, similar in pricing to WaveNet (e.g., 1M free characters/month, then $16/1M characters).
    These values should be verified against the latest Google Cloud TTS pricing documentation.
    """
    # Defines the STANDARD enum member, representing cost-efficient standard voices,
    # with an associated string value "standard". E.g., $4/1M chars after 4M free monthly.
    STANDARD = "standard"
    # Defines the WAVENET enum member, representing high-quality WaveNet voices,
    # with an associated string value "wavenet". E.g., $16/1M chars after 1M free monthly.
    WAVENET = "wavenet"
    # Defines the NEURAL2 enum member, representing the latest Neural2 voices,
    # with an associated string value "neural2". E.g., $16/1M chars after 1M free monthly.
    NEURAL2 = "neural2"

    # Defines a method to retrieve the cost per million characters for the voice quality tier.
    def get_cost_per_million_characters(self) -> float:
        """
        Returns the cost in USD per million characters for this voice quality tier,
        applicable after the free tier is exhausted.

        Returns:
            float: Cost in USD per million characters.
        """
        # Check if the current enum instance (self) is VoiceQuality.STANDARD.
        if self == VoiceQuality.STANDARD:
            # Return the cost (4.0 USD) for standard voices per million characters.
            return 4.0
        # Handle cases where the voice quality is not STANDARD (i.e., WAVENET or NEURAL2).
        else: # WAVENET or NEURAL2
            # Return the cost (16.0 USD) for premium voices (WaveNet/Neural2) per million characters.
            return 16.0

    # Defines a method to retrieve the monthly free tier character limit for the voice quality.
    def get_free_tier_character_limit(self) -> int:
        """
        Returns the monthly free tier character limit for this voice quality.

        Returns:
            int: Number of free characters allocated per month for this tier.
        """
        # Check if the current enum instance (self) is VoiceQuality.STANDARD.
        if self == VoiceQuality.STANDARD:
            # Return the monthly free character limit (4,000,000) for standard voices.
            return 4_000_000
        # Handle cases where the voice quality is not STANDARD (i.e., WAVENET or NEURAL2).
        else: # WAVENET or NEURAL2
            # Return the monthly free character limit (1,000,000) for premium voices.
            return 1_000_000

    # Defines a static method to determine VoiceQuality from a Google Cloud TTS voice name string.
    @staticmethod
    # Specifies the method signature: takes a 'voice_name' string and returns a 'VoiceQuality' enum member.
    def from_voice_name(voice_name: str) -> 'VoiceQuality':
        """
        Determines the VoiceQuality tier from a Google Cloud TTS voice name.

        Args:
            voice_name: The name of the voice (e.g., "en-US-Standard-C", "en-US-Wavenet-A", "en-US-Neural2-F").

        Returns:
            VoiceQuality: The corresponding quality tier.

        Raises:
            ValueError: If the voice name does not match a known quality pattern.
        """
        # Perform a substring check to determine if "Neural2" is present in the 'voice_name' string.
        if "Neural2" in voice_name:
            # Return the VoiceQuality.NEURAL2 enum member if "Neural2" is found.
            return VoiceQuality.NEURAL2
        # Else, perform a substring check to determine if "Wavenet" is present in the 'voice_name' string.
        elif "Wavenet" in voice_name: # Note: Google often uses "Wavenet" (capital W) in names.
            # Return the VoiceQuality.WAVENET enum member if "Wavenet" is found.
            return VoiceQuality.WAVENET
        # Else, perform a substring check to determine if "Standard" is present in the 'voice_name' string.
        elif "Standard" in voice_name:
            # Return the VoiceQuality.STANDARD enum member if "Standard" is found.
            return VoiceQuality.STANDARD
        # Handle cases where the voice name does not match any of the known quality patterns.
        else:
            # This indicates an unknown or new voice type not covered by the enum logic.
            # Log a warning message indicating inability to determine quality and the defaulting behavior.
            logger.warning(f"Could not determine voice quality from voice name: {voice_name}. Defaulting to STANDARD.")
            # Default to and return VoiceQuality.STANDARD for unrecognized voice name patterns.
            return VoiceQuality.STANDARD


In [ ]:
# Defines the 'TTSRequest' dataclass, a structured representation for Text-to-Speech (TTS) requests,
# incorporating comprehensive validation, cost optimization metadata, and tracking identifiers.
@dataclass
class TTSRequest:
    """
    Structured Text-to-Speech (TTS) request with comprehensive validation,
    cost optimization metadata, and tracking identifiers.

    This dataclass encapsulates all parameters required for TTS synthesis.
    It includes built-in validation in `__post_init__` to ensure data integrity
    before processing.
    """
    # Defines the 'text' field: The primary text content to be synthesized into speech. Must be a non-empty string.
    text: str
    # Defines the 'voice_name' field: Optional specific voice name (e.g., "en-US-Neural2-A"). Defaults to None.
    voice_name: Optional[str] = None
    # Defines the 'language_code' field: Language code in BCP-47 format (e.g., 'en-US'). Defaults to 'en-US'.
    language_code: str = "en-US"
    # Defines the 'quality_tier' field: Desired voice quality tier, affecting audio and cost. Defaults to VoiceQuality.STANDARD.
    quality_tier: VoiceQuality = VoiceQuality.STANDARD
    # Defines the 'ssml_enabled' field: Flag indicating if 'text' contains SSML tags. Defaults to False.
    ssml_enabled: bool = False
    # Defines the 'speaking_rate' field: Speaking rate/speed (1.0 is normal). Defaults to 1.0.
    speaking_rate: float = 1.0
    # Defines the 'pitch' field: Pitch adjustment (0.0 is normal). Defaults to 0.0.
    pitch: float = 0.0
    # Defines the 'audio_encoding' field: Audio encoding format. Defaults to MP3.
    audio_encoding: texttospeech.AudioEncoding = texttospeech.AudioEncoding.MP3

    # --- Internal fields, typically set by the orchestrator ---
    # Defines 'cache_key': Generated cache key for this request. Not part of public representation. Defaults to None.
    cache_key: Optional[str] = field(default=None, repr=False)
    # Defines 'estimated_cost': Estimated USD cost for this request. Not part of public representation. Defaults to 0.0.
    estimated_cost: float = field(default=0.0, repr=False)
    # Defines 'priority': Priority for batch processing (1=highest). Defaults to 5.
    priority: int = field(default=5)
    # Defines 'request_id': Unique identifier for this request, generated via UUID.
    request_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    # Defines 'created_at': Timestamp (UTC) of request object creation.
    created_at: datetime = field(default_factory=lambda: datetime.now(timezone.utc))

    # Defines the post-initialization hook method, automatically called after the dataclass instance is created.
    def __post_init__(self):
        """
        Performs validation on all fields after the dataclass instance is initialized.
        Ensures that the request data is sane and meets predefined constraints.

        Raises:
            ValidationError: If any field contains invalid data according to the defined rules.
            TypeError: If any field has an incorrect data type.
        """
        # Validate 'text': ensure it is a string and not empty or only whitespace.
        if not isinstance(self.text, str) or not self.text.strip():
            # Construct an error message for invalid 'text'.
            error_msg = f"Request {self.request_id}: Text must be a non-empty string."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)
        # Define the maximum allowed length for the 'text' field in bytes (UTF-8 encoded).
        max_length = 5000 # General limit, can be more specific if API docs state so.
        # Check if the UTF-8 byte length of 'text' exceeds the defined 'max_length'.
        if len(self.text.encode('utf-8')) > max_length: # Check byte length as per Google's quota
            # Construct an error message for 'text' length exceeding the maximum.
            error_msg = f"Request {self.request_id}: Text length ({len(self.text.encode('utf-8'))} bytes) exceeds maximum allowed ({max_length} bytes)."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'language_code': ensure it conforms to the BCP-47 format by calling the internal validation method.
        if not self._is_valid_language_code(self.language_code):
            # Construct an error message for an invalid 'language_code' format.
            error_msg = f"Request {self.request_id}: Invalid language code format: '{self.language_code}'. Must be BCP-47."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'quality_tier': ensure it is an instance of the VoiceQuality enum.
        if not isinstance(self.quality_tier, VoiceQuality):
            # This check is for runtime safety, though type hinting should ideally catch this.
            # Construct an error message for an invalid 'quality_tier' type.
            error_msg = f"Request {self.request_id}: quality_tier must be a VoiceQuality enum value."
            # Raise a TypeError with the constructed message (as it's a type mismatch).
            raise TypeError(error_msg)

        # Validate 'priority': ensure it is an integer and within the defined range [1, 10].
        if not isinstance(self.priority, int) or not (1 <= self.priority <= 10):
            # Construct an error message for an invalid 'priority' value.
            error_msg = f"Request {self.request_id}: Priority must be an integer between 1 and 10, got {self.priority}."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'text' content if 'ssml_enabled' is True, using the internal SSML validation method.
        if self.ssml_enabled and not self._is_valid_ssml(self.text):
            # Construct an error message for invalid or malformed SSML in 'text'.
            error_msg = f"Request {self.request_id}: Invalid or malformed SSML markup detected in text when ssml_enabled is True."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'speaking_rate': ensure it is a number (int or float) and within Google's typical accepted range [0.25, 4.0].
        if not isinstance(self.speaking_rate, (int, float)) or not (0.25 <= self.speaking_rate <= 4.0):
            # Construct an error message for an invalid 'speaking_rate' value.
            error_msg = f"Request {self.request_id}: Speaking rate must be a float between 0.25 and 4.0, got {self.speaking_rate}."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'pitch': ensure it is a number (int or float) and within Google's typical accepted range [-20.0, 20.0].
        if not isinstance(self.pitch, (int, float)) or not (-20.0 <= self.pitch <= 20.0):
            # Construct an error message for an invalid 'pitch' value.
            error_msg = f"Request {self.request_id}: Pitch must be a float between -20.0 and 20.0, got {self.pitch}."
            # Raise a ValidationError with the constructed message.
            raise ValidationError(error_msg)

        # Validate 'audio_encoding': ensure it is an instance of the texttospeech.AudioEncoding enum.
        if not isinstance(self.audio_encoding, texttospeech.AudioEncoding):
            # Construct an error message for an invalid 'audio_encoding' type.
            error_msg = f"Request {self.request_id}: audio_encoding must be a texttospeech.AudioEncoding enum value."
            # Raise a TypeError with the constructed message (as it's a type mismatch).
            raise TypeError(error_msg)

    # Defines an internal helper method to validate the BCP-47 format of a language code string.
    def _is_valid_language_code(self, code: str) -> bool:
        """
        Validates if the given language code string conforms to the BCP-47 standard format.
        Example: 'en-US', 'fr-FR', 'es', 'zh-CN'.

        Args:
            code: The language code string to validate.

        Returns:
            bool: True if the code is a valid BCP-47 format, False otherwise.
        """
        # Define the regular expression pattern for BCP-47 language code validation.
        # This pattern is a common approximation for language[-script][-region] formats.
        pattern = r'^[a-zA-Z]{2,3}(-[a-zA-Z]{2,4})?(-[a-zA-Z0-9]{2,8})*$'
        # Check if 'code' is a string and if it fully matches the defined 'pattern'.
        # The 'bool()' conversion ensures a True/False return value from 're.fullmatch' result.
        return isinstance(code, str) and bool(re.fullmatch(pattern, code))

    # Defines an internal helper method to perform basic validation on SSML text.
    def _is_valid_ssml(self, text: str) -> bool:
        """
        Performs basic validation on SSML text to catch common malformations.
        Checks for the presence and basic balance of <speak> tags.
        Note: This is not a full XML/SSML parser. For production, a more robust
        SSML validation library might be considered if complex SSML is used.

        Args:
            text: The SSML string to validate.

        Returns:
            bool: True if the SSML appears to be minimally valid, False otherwise.
        """
        # Check if the trimmed 'text' starts with '<speak>' and ends with '</speak>'.
        if not (text.strip().startswith('<speak>') and text.strip().endswith('</speak>')):
            # Log a debug message indicating the SSML validation failure due to missing/misplaced <speak> tags.
            logger.debug(f"Request {self.request_id}: SSML validation failed: Missing or misplaced <speak> tags.")
            # Return False as the SSML is invalid.
            return False

        # Perform a basic count of common opening SSML tags for a simplified balance check.
        open_tags = text.count('<speak>') + text.count('<voice') + text.count('<prosody') + text.count('<say-as') + text.count('<break')
        # Perform a basic count of common closing SSML tags for the simplified balance check.
        # Note: Self-closing tags like <break /> are not explicitly handled differently in this simplified count.
        close_tags = text.count('</speak>') + text.count('</voice>') + text.count('</prosody>') + text.count('</say-as>')

        # Check if the count of closing tags is greater than opening tags, which indicates a definite malformation.
        if open_tags < close_tags : # More closing than opening is definitely wrong
             # Log a debug message indicating the SSML validation failure due to mismatched tag counts.
             logger.debug(f"Request {self.request_id}: SSML validation failed: Mismatched tag counts (open: {open_tags}, close: {close_tags}).")
             # Return False as the SSML is invalid.
             return False
        # Note: open_tags > close_tags can occur with unclosed tags or self-closing tags not perfectly accounted for by this heuristic.

        # Return True if the SSML passes these basic validation checks.
        return True # Passes basic checks.

    # Defines a method to calculate the billable character count for the TTS request.
    def get_billable_character_count(self) -> int:
        """
        Calculates the number of characters that will be billed by Google Cloud TTS.
        If SSML is enabled, tags are typically not counted towards the billable characters.
        The exact rules should be confirmed with Google Cloud documentation.

        Returns:
            int: The count of billable characters.
        """
        # Check if SSML processing is disabled for this request.
        if not self.ssml_enabled:
            # If SSML is not enabled, the entire length of the 'text' is billable.
            return len(self.text)

        # If SSML is enabled, remove all SSML tags from the 'text' to count only the speakable content.
        # This regular expression aims to remove any content enclosed in <...>.
        plain_text = re.sub(r'<[^>]+>', '', self.text)
        # Return the length of the SSML-stripped plain text, which represents the billable characters.
        return len(plain_text)


In [ ]:
# Defines the 'CostMetrics' dataclass, responsible for managing comprehensive, thread-safe
# cost tracking and analysis for Text-to-Speech (TTS) usage.
@dataclass
class CostMetrics:
    """
    Manages comprehensive, thread-safe cost tracking and analysis for TTS usage.

    This class maintains detailed metrics related to character processing,
    cost accumulation, cache performance, and request statistics. All update
    operations are thread-safe using an internal lock.
    """
    # Defines 'characters_processed': Total characters from successful synthesis requests. Initialized to 0.
    characters_processed: int = 0
    # Defines 'standard_voice_usage': Characters processed using STANDARD quality voices. Initialized to 0.
    standard_voice_usage: int = 0
    # Defines 'premium_voice_usage': Characters processed using premium (WaveNet, Neural2) voices. Initialized to 0.
    premium_voice_usage: int = 0
    # Defines 'estimated_monthly_cost': Running total of estimated costs in USD. Initialized to 0.0.
    estimated_monthly_cost: float = 0.0
    # Defines 'cache_hit_rate': Cache hit rate as a decimal (e.g., 0.75 for 75%). Initialized to 0.0.
    cache_hit_rate: float = 0.0
    # Defines 'successful_requests': Count of successful synthesis requests. Initialized to 0.
    successful_requests: int = 0
    # Defines 'failed_requests': Count of failed synthesis requests. Initialized to 0.
    failed_requests: int = 0
    # Defines 'cache_hits': Total times a result was served from cache. Initialized to 0.
    cache_hits: int = 0
    # Defines 'cache_misses': Total times a result was not found in cache. Initialized to 0.
    cache_misses: int = 0
    # Defines 'avg_response_time_ms': Rolling average response time for successful synthesis (ms). Initialized to 0.0.
    avg_response_time_ms: float = 0.0
    # Defines 'last_reset_utc': Timestamp (UTC) of the last monthly metrics reset. Defaults to current UTC time.
    last_reset_utc: datetime = field(default_factory=lambda: datetime.now(timezone.utc))
    # Defines '_lock': Internal threading.Lock for thread-safe updates. Not part of public API, not initialized by user.
    _lock: threading.Lock = field(default_factory=threading.Lock, init=False, repr=False)

    # Defines a method to thread-safely update metrics after a successful synthesis operation.
    def update_on_synthesis_success(self, character_count: int, quality_tier: VoiceQuality, cost: float, response_time_ms: float) -> None:
        """
        Thread-safely updates metrics after a successful synthesis operation.

        Args:
            character_count: Number of billable characters processed in the request.
            quality_tier: The VoiceQuality tier used for the synthesis.
            cost: Estimated cost incurred for this synthesis request.
            response_time_ms: Time taken for the synthesis operation in milliseconds.
        """
        # Assert that the provided 'character_count' is non-negative.
        assert character_count >= 0, "Character count for metric update cannot be negative."
        # Assert that the provided 'cost' is non-negative.
        assert cost >= 0.0, "Cost for metric update cannot be negative."
        # Assert that the provided 'response_time_ms' is non-negative.
        assert response_time_ms >= 0.0, "Response time for metric update cannot be negative."

        # Acquire the internal thread lock to ensure atomic updates to shared metric fields.
        with self._lock:
            # Increment the total 'characters_processed' by the 'character_count' of the current request.
            self.characters_processed += character_count
            # Check if the 'quality_tier' used for the synthesis was STANDARD.
            if quality_tier == VoiceQuality.STANDARD:
                # Increment 'standard_voice_usage' by the 'character_count'.
                self.standard_voice_usage += character_count
            # Handle cases where a premium quality tier (WaveNet or Neural2) was used.
            else:
                # Increment 'premium_voice_usage' by the 'character_count'.
                self.premium_voice_usage += character_count
            # Add the 'cost' of the current synthesis to the 'estimated_monthly_cost'.
            self.estimated_monthly_cost += cost
            # Increment the count of 'successful_requests'.
            self.successful_requests += 1

            # Calculate the total number of completed requests for averaging response time.
            total_completed_requests = self.successful_requests # Only average successful ones for API perf
            # Check if this is the first successful request.
            if total_completed_requests == 1: # First successful request
                 # Set the 'avg_response_time_ms' directly to the 'response_time_ms' of this first request.
                 self.avg_response_time_ms = response_time_ms
            # Handle subsequent successful requests to update the rolling average.
            else:
                 # Update 'avg_response_time_ms' using the rolling average formula: NewAvg = ((OldAvg * (N-1)) + NewValue) / N.
                 self.avg_response_time_ms = ((self.avg_response_time_ms * (total_completed_requests - 1)) + response_time_ms) / total_completed_requests

    # Defines a method to thread-safely update cache performance metrics.
    def update_cache_metrics(self, cache_hit: bool) -> None:
        """
        Thread-safely updates cache performance metrics (hits, misses, hit rate).

        Args:
            cache_hit: True if the request was served from cache, False otherwise.
        """
        # Acquire the internal thread lock to ensure atomic updates to shared cache metric fields.
        with self._lock:
            # Check if the cache lookup resulted in a hit.
            if cache_hit:
                # Increment the 'cache_hits' counter.
                self.cache_hits += 1
            # Handle cases where the cache lookup resulted in a miss.
            else:
                # Increment the 'cache_misses' counter.
                self.cache_misses += 1

            # Calculate the total number of cache lookups (hits + misses).
            total_cache_lookups = self.cache_hits + self.cache_misses
            # Check if there have been any cache lookups to avoid division by zero.
            if total_cache_lookups > 0:
                # Recalculate 'cache_hit_rate' as the ratio of hits to total lookups.
                self.cache_hit_rate = self.cache_hits / total_cache_lookups
            # Handle the case where there have been no cache lookups yet.
            else:
                # Set 'cache_hit_rate' to 0.0 to avoid division by zero.
                self.cache_hit_rate = 0.0

    # Defines a method to thread-safely record a failed synthesis attempt.
    def record_synthesis_failure(self) -> None:
        """
        Thread-safely records a failed synthesis attempt.
        """
        # Acquire the internal thread lock to ensure atomic update to the 'failed_requests' counter.
        with self._lock:
            # Increment the 'failed_requests' counter.
            self.failed_requests += 1

    # Defines a method to return a thread-safe snapshot of all current metric values.
    def get_current_metrics_snapshot(self) -> Dict[str, Any]:
        """
        Returns a thread-safe snapshot of all current metric values.
        This is useful for reporting or analysis without direct mutable access.

        Returns:
            Dict[str, Any]: A dictionary containing the current state of all metrics.
        """
        # Acquire the internal thread lock to ensure a consistent and atomic read of all metric fields.
        with self._lock:
            # Create and return a dictionary containing copies of all current metric values.
            # This prevents external modification of the internal metric state.
            return {
                'characters_processed': self.characters_processed,
                'standard_voice_usage': self.standard_voice_usage,
                'premium_voice_usage': self.premium_voice_usage,
                'estimated_monthly_cost': self.estimated_monthly_cost,
                'cache_hit_rate': self.cache_hit_rate,
                'successful_requests': self.successful_requests,
                'failed_requests': self.failed_requests,
                'cache_hits': self.cache_hits,
                'cache_misses': self.cache_misses,
                'avg_response_time_ms': self.avg_response_time_ms,
                # Convert the 'last_reset_utc' datetime object to an ISO 8601 string format for serialization.
                'last_reset_utc': self.last_reset_utc.isoformat()
            }

    # Defines a method to thread-safely reset metrics typically tracked on a monthly basis.
    def reset_monthly_metrics(self) -> None:
        """
        Thread-safely resets metrics that are typically tracked on a monthly basis
        (e.g., usage counts, estimated cost). Cache performance and overall
        request counts might be kept longer or reset differently.
        """
        # Acquire the internal thread lock to ensure an atomic reset of monthly metric fields.
        with self._lock:
            # Reset 'standard_voice_usage' to 0.
            self.standard_voice_usage = 0
            # Reset 'premium_voice_usage' to 0.
            self.premium_voice_usage = 0
            # Reset 'estimated_monthly_cost' to 0.0.
            self.estimated_monthly_cost = 0.0
            # Update 'last_reset_utc' to the current UTC date and time.
            self.last_reset_utc = datetime.now(timezone.utc)
            # Log an informational message indicating that monthly metrics have been reset.
            logger.info("Monthly cost metrics have been reset.")
            # Note: Other metrics like 'characters_processed', cache stats, 'avg_response_time_ms'
            # are not reset here as they might be tracked over different periods.


In [ ]:
# Defines the 'CircuitBreaker' class, implementing the Circuit Breaker reliability pattern
# to enhance resilience against failing external API calls.
class CircuitBreaker:
    """
    Implements the Circuit Breaker pattern to enhance resilience against failing external API calls.

    When an external service (like Google Cloud TTS API) experiences issues,
    the circuit breaker prevents an application from repeatedly trying an operation
    that is likely to fail. This saves resources and avoids cascading failures.

    States:
    - CLOSED: Normal operation. Calls pass through. Failures are counted.
    - OPEN: After a threshold of failures, the circuit opens. Calls are rejected immediately
            without attempting the operation. After a timeout, transitions to HALF_OPEN.
    - HALF_OPEN: Allows a limited number of test calls. If successful, transitions to CLOSED.
                 If it fails, transitions back to OPEN.
    """
    # Defines a class constant representing the 'CLOSED' state of the circuit breaker.
    STATE_CLOSED = "CLOSED"
    # Defines a class constant representing the 'OPEN' state of the circuit breaker.
    STATE_OPEN = "OPEN"
    # Defines a class constant representing the 'HALF_OPEN' state of the circuit breaker.
    STATE_HALF_OPEN = "HALF_OPEN"

    # Defines the constructor for the CircuitBreaker class.
    def __init__(self, failure_threshold: int = 5, recovery_timeout_seconds: float = 60.0, half_open_attempt_limit: int = 1):
        """
        Initializes the CircuitBreaker.

        Args:
            failure_threshold: Number of consecutive failures to trigger opening the circuit.
            recovery_timeout_seconds: Time in seconds the circuit stays OPEN before transitioning to HALF_OPEN.
            half_open_attempt_limit: Number of allowed attempts in HALF_OPEN state.
        """
        # Assign the 'failure_threshold' argument to the instance variable 'self.failure_threshold'.
        self.failure_threshold = failure_threshold
        # Assign the 'recovery_timeout_seconds' argument to the instance variable 'self.recovery_timeout_seconds'.
        self.recovery_timeout_seconds = recovery_timeout_seconds
        # Assign the 'half_open_attempt_limit' argument to the instance variable 'self.half_open_attempt_limit'.
        self.half_open_attempt_limit = half_open_attempt_limit

        # Initialize the internal counter for consecutive failures to 0.
        self._failure_count = 0
        # Initialize the timestamp of the last recorded failure to None, as no failures have occurred yet.
        self._last_failure_time_utc: Optional[datetime] = None
        # Initialize the current state of the circuit breaker to 'STATE_CLOSED'.
        self._state = CircuitBreaker.STATE_CLOSED
        # Initialize the counter for attempts made in the 'HALF_OPEN' state to 0.
        self._half_open_attempts = 0
        # Initialize a threading.Lock object for ensuring thread-safe modifications to the circuit breaker's state.
        self._lock = threading.Lock()
        # Obtain a logger instance specific to this CircuitBreaker class instance.
        self._logger = logging.getLogger(f"{__name__}.CircuitBreaker")

    # Defines a property 'state' to get the current state of the circuit breaker in a thread-safe manner.
    @property
    # Specifies the method signature for the state getter.
    def state(self) -> str:
        """Returns the current state of the circuit breaker."""
        # Acquire the internal thread lock to ensure thread-safe access to the '_state' attribute.
        with self._lock:
            # Return the current value of the '_state' attribute.
            return self._state

    # Defines an internal method to determine if a call can be attempted based on the current circuit breaker state.
    def _can_attempt_call(self) -> bool:
        """Determines if a call can be attempted based on the current state."""
        # Check if the current state of the circuit breaker is 'STATE_CLOSED'.
        if self._state == CircuitBreaker.STATE_CLOSED:
            # If CLOSED, always allow the call attempt.
            return True
        # Check if the current state of the circuit breaker is 'STATE_OPEN'.
        if self._state == CircuitBreaker.STATE_OPEN:
            # Check if a last failure time is recorded and if the recovery timeout has elapsed.
            if self._last_failure_time_utc and \
               (datetime.now(timezone.utc) - self._last_failure_time_utc).total_seconds() >= self.recovery_timeout_seconds:
                # If timeout elapsed, transition the state to 'STATE_HALF_OPEN'.
                self._state = CircuitBreaker.STATE_HALF_OPEN
                # Reset the half-open attempt counter for the new HALF_OPEN phase.
                self._half_open_attempts = 0
                # Log the state transition to HALF_OPEN.
                self._logger.info("Circuit breaker transitioned to HALF_OPEN state.")
                # Allow the first attempt in the new HALF_OPEN state.
                return True
            # If still in OPEN state and timeout has not elapsed, do not allow the call attempt.
            return False
        # Check if the current state of the circuit breaker is 'STATE_HALF_OPEN'.
        if self._state == CircuitBreaker.STATE_HALF_OPEN:
            # Allow the call attempt if the number of half-open attempts is less than the defined limit.
            return self._half_open_attempts < self.half_open_attempt_limit
        # This path should ideally not be reached if states are managed correctly; return False as a safeguard.
        return False

    # Defines an asynchronous method to execute a function under circuit breaker protection.
    async def execute_async(self, func: Callable[..., Any], *args: Any, **kwargs: Any) -> Any:
        """
        Executes an asynchronous function under circuit breaker protection.

        Args:
            func: The asynchronous function to execute.
            *args: Positional arguments for the function.
            **kwargs: Keyword arguments for the function.

        Returns:
            The result of the function if successful.

        Raises:
            APICallError: If the circuit is OPEN or if the function fails while in HALF_OPEN
                               (causing a transition back to OPEN), or if the underlying call fails.
            Exception: Any exception raised by the wrapped function if it's not an APICallError.
        """
        # Acquire the internal thread lock to check and potentially update the circuit breaker's state atomically.
        with self._lock:
            # Determine if the call can be attempted based on the current state.
            if not self._can_attempt_call():
                # If the call cannot be attempted (e.g., circuit is OPEN or HALF_OPEN attempts exhausted), log a warning.
                self._logger.warning(f"Circuit breaker is {self._state}. Call to {func.__name__} rejected.")
                # Raise an APICallError indicating that the call was rejected by the circuit breaker.
                raise APICallError(f"Circuit breaker is {self._state}. Call rejected.")

            # Check if the circuit breaker is currently in the 'STATE_HALF_OPEN'.
            if self._state == CircuitBreaker.STATE_HALF_OPEN:
                # If in HALF_OPEN state, increment the counter for half-open attempts.
                self._half_open_attempts += 1

        # The lock is released before executing the (potentially long-running) wrapped function 'func'.
        # Begin a try block to handle potential exceptions during the execution of 'func'.
        try:
            # Await the execution of the provided asynchronous function 'func' with its arguments.
            result = await func(*args, **kwargs)
            # If 'func' executes successfully, record this success with the circuit breaker.
            self.record_success()
            # Return the result obtained from the successful execution of 'func'.
            return result
        # Catch any exception 'e' that occurs during the execution of 'func'.
        except Exception as e:
            # If 'func' fails, record this failure with the circuit breaker.
            self.record_failure()
            # Log an error message detailing the failure recorded by the circuit breaker.
            self._logger.error(f"Circuit breaker recorded failure for {func.__name__}: {e}", exc_info=True)
            # Check if the caught exception 'e' is already an instance of APICallError.
            if isinstance(e, APICallError):
                # If it is, re-raise the original APICallError.
                raise
            # If 'e' is not an APICallError, wrap it in a new APICallError and raise it, preserving the original exception.
            raise APICallError(f"Call to {func.__name__} failed: {e}") from e

    # Defines a method to record a successful call, potentially closing or resetting the circuit.
    def record_success(self) -> None:
        """Records a successful call, potentially closing or resetting the circuit."""
        # Acquire the internal thread lock to ensure atomic updates to the circuit breaker's state.
        with self._lock:
            # Check if the circuit breaker is currently in the 'STATE_HALF_OPEN'.
            if self._state == CircuitBreaker.STATE_HALF_OPEN:
                # If in HALF_OPEN and the call was successful, transition the state to 'STATE_CLOSED'.
                self._state = CircuitBreaker.STATE_CLOSED
                # Reset the consecutive failure count to 0.
                self._failure_count = 0
                # Reset the half-open attempt counter to 0.
                self._half_open_attempts = 0
                # Log the state transition to CLOSED.
                self._logger.info("Circuit breaker transitioned to CLOSED state after successful HALF_OPEN attempt.")
            # Check if the circuit breaker is currently in the 'STATE_CLOSED'.
            elif self._state == CircuitBreaker.STATE_CLOSED:
                # If in CLOSED state and there were previous intermittent failures (failure_count > 0), reset the count.
                if self._failure_count > 0:
                    # Reset the consecutive failure count to 0 upon any successful call in CLOSED state.
                    self._failure_count = 0
                    # Log a debug message about resetting the failure count in CLOSED state.
                    self._logger.debug("Circuit breaker failure count reset in CLOSED state due to success.")

    # Defines a method to record a failed call, potentially opening the circuit.
    def record_failure(self) -> None:
        """Records a failed call, potentially opening the circuit."""
        # Acquire the internal thread lock to ensure atomic updates to the circuit breaker's state.
        with self._lock:
            # Update the timestamp of the last recorded failure to the current UTC time.
            self._last_failure_time_utc = datetime.now(timezone.utc)
            # Check if the circuit breaker is currently in the 'STATE_HALF_OPEN'.
            if self._state == CircuitBreaker.STATE_HALF_OPEN:
                # If a failure occurs in HALF_OPEN state, transition back to 'STATE_OPEN'.
                self._state = CircuitBreaker.STATE_OPEN
                # Set failure count to threshold to ensure it remains open for the recovery timeout period.
                self._failure_count = self.failure_threshold
                # Log a warning about the transition back to OPEN state due to failure in HALF_OPEN.
                self._logger.warning("Circuit breaker transitioned back to OPEN state due to failure in HALF_OPEN.")
            # Check if the circuit breaker is currently in the 'STATE_CLOSED'.
            elif self._state == CircuitBreaker.STATE_CLOSED:
                # If in CLOSED state, increment the consecutive failure count.
                self._failure_count += 1
                # Log a debug message about the incremented failure count in CLOSED state.
                self._logger.debug(f"Circuit breaker failure count incremented to {self._failure_count} in CLOSED state.")
                # Check if the failure count has reached or exceeded the defined failure threshold.
                if self._failure_count >= self.failure_threshold:
                    # If the threshold is met, transition the state to 'STATE_OPEN'.
                    self._state = CircuitBreaker.STATE_OPEN
                    # Log a warning about the transition to OPEN state due to exceeding the failure threshold.
                    self._logger.warning(f"Circuit breaker transitioned to OPEN state after {self._failure_count} failures.")


In [ ]:
# Defines the 'AdaptiveRateLimiter' class, which implements an adaptive rate limiting mechanism
# that adjusts request delays based on API response patterns.
class AdaptiveRateLimiter:
    """
    Implements an adaptive rate limiter that adjusts request delays based on API
    response patterns (successes, failures, rate limit errors).

    This helps in dynamically managing request rates to avoid hitting API quotas
    and to adapt to changing server load conditions.
    """
    # Defines the constructor for the AdaptiveRateLimiter class.
    def __init__(self,
                 initial_delay_seconds: float = 0.1,
                 max_delay_seconds: float = 5.0,
                 min_delay_seconds: float = 0.05, # Minimum possible delay
                 success_streak_for_decrease: int = 10, # Consecutive successes to decrease delay
                 decrease_factor: float = 0.9, # Factor to decrease delay by
                 failure_increase_factor: float = 1.5, # Factor to increase delay on general failure
                 rate_limit_error_increase_factor: float = 2.0): # Factor for rate limit errors
        """
        Initializes the AdaptiveRateLimiter.

        Args:
            initial_delay_seconds: Starting delay between requests in seconds.
            max_delay_seconds: Maximum possible delay between requests.
            min_delay_seconds: Minimum possible delay, even after many successes.
            success_streak_for_decrease: Number of consecutive successes before attempting to decrease delay.
            decrease_factor: Multiplicative factor to decrease delay (e.g., 0.9 for 10% decrease).
            failure_increase_factor: Multiplicative factor to increase delay on a general failure.
            rate_limit_error_increase_factor: Multiplicative factor for more aggressive delay increase on rate limit errors.
        """
        # Assign the 'min_delay_seconds' argument to the instance variable 'self.min_delay_seconds'.
        self.min_delay_seconds = min_delay_seconds
        # Assign the 'max_delay_seconds' argument to the instance variable 'self.max_delay_seconds'.
        self.max_delay_seconds = max_delay_seconds
        # Initialize 'self.current_delay_seconds', ensuring it's not below 'min_delay_seconds'.
        self.current_delay_seconds = max(min_delay_seconds, initial_delay_seconds)

        # Initialize the internal counter for consecutive successful requests to 0.
        self._success_streak = 0
        # Initialize the internal counter for consecutive failed requests to 0 (primarily for monitoring).
        self._failure_streak = 0

        # Assign 'success_streak_for_decrease' to an internal instance variable.
        self._success_streak_for_decrease = success_streak_for_decrease
        # Assign 'decrease_factor' to an internal instance variable.
        self._decrease_factor = decrease_factor
        # Assign 'failure_increase_factor' to an internal instance variable.
        self._failure_increase_factor = failure_increase_factor
        # Assign 'rate_limit_error_increase_factor' to an internal instance variable.
        self._rate_limit_error_increase_factor = rate_limit_error_increase_factor

        # Initialize the timestamp of the last request using a monotonic clock (nanoseconds) for accurate interval measurement.
        self._last_request_time_ns = time.monotonic_ns()
        # Initialize an asyncio.Lock object for ensuring thread-safe (in async context) updates to the limiter's state.
        self._lock = asyncio.Lock()
        # Obtain a logger instance specific to this AdaptiveRateLimiter class instance.
        self._logger = logging.getLogger(f"{__name__}.AdaptiveRateLimiter")

    # Defines an asynchronous method to acquire permission to proceed, waiting if necessary based on rate limits.
    async def acquire(self) -> None:
        """
        Asynchronously waits for the appropriate time based on the current rate limiting state.
        This method should be awaited before making an external API call.
        """
        # Declare 'wait_time_ns' to store the calculated wait duration in nanoseconds.
        wait_time_ns: int # Type hint for clarity

        # Asynchronously acquire the internal lock to ensure atomic calculation of wait time and update of last request time.
        async with self._lock:
            # Get the current time using a monotonic clock in nanoseconds for precise interval calculation.
            current_time_ns = time.monotonic_ns()
            # Calculate the time elapsed in nanoseconds since the last request was made.
            time_since_last_request_ns = current_time_ns - self._last_request_time_ns
            # Calculate the required delay in nanoseconds based on the current delay setting.
            required_delay_ns = int(self.current_delay_seconds * 1_000_000_000)

            # Initialize 'wait_time_ns' to 0, assuming no wait is needed initially.
            wait_time_ns = 0
            # Check if the time since the last request is less than the required delay.
            if time_since_last_request_ns < required_delay_ns:
                # If so, calculate the necessary wait time in nanoseconds.
                wait_time_ns = required_delay_ns - time_since_last_request_ns

            # Update the last request time to the current time plus any enforced wait time.
            # This effectively reserves the slot for the current request and ensures subsequent calls respect this one.
            self._last_request_time_ns = current_time_ns + wait_time_ns

        # Perform the actual wait (sleep) outside the lock to avoid holding it during the sleep period.
        # Check if a non-zero wait time was calculated.
        if wait_time_ns > 0:
            # Convert the wait time from nanoseconds to seconds for asyncio.sleep.
            wait_time_seconds = wait_time_ns / 1_000_000_000
            # Log a debug message indicating the rate limiter is waiting.
            self._logger.debug(f"Rate limiter waiting for {wait_time_seconds:.3f} seconds. Current delay: {self.current_delay_seconds:.3f}s.")
            # Asynchronously pause execution for the calculated 'wait_time_seconds'.
            await asyncio.sleep(wait_time_seconds)

    # Defines an asynchronous method to record a successful API request and adjust rate limiting delay.
    async def record_success(self) -> None:
        """
        Records a successful API request and potentially adjusts the rate limiting delay downwards.
        """
        # Asynchronously acquire the internal lock to ensure atomic updates to rate limiter state.
        async with self._lock:
            # Increment the consecutive success streak counter.
            self._success_streak += 1
            # Reset the consecutive failure streak counter upon a successful request.
            self._failure_streak = 0

            # Check if the success streak has reached the threshold required for decreasing the delay.
            if self._success_streak >= self._success_streak_for_decrease:
                # Calculate the new delay by applying the decrease factor.
                new_delay = self.current_delay_seconds * self._decrease_factor
                # Update the current delay, ensuring it does not fall below the defined minimum delay.
                self.current_delay_seconds = max(self.min_delay_seconds, new_delay)
                # Reset the success streak counter after adjusting the delay.
                self._success_streak = 0
                # Log a debug message indicating the delay decrease.
                self._logger.debug(f"Rate limiter delay decreased to {self.current_delay_seconds:.3f}s after success streak.")

    # Defines an asynchronous method to record a failed API request and adjust rate limiting delay.
    async def record_failure(self, is_rate_limit_error: bool = False) -> None:
        """
        Records a failed API request and adjusts the rate limiting delay upwards.
        More aggressive increase for specific rate limit errors.

        Args:
            is_rate_limit_error: True if the failure was due to an explicit rate limiting error from the API.
        """
        # Asynchronously acquire the internal lock to ensure atomic updates to rate limiter state.
        async with self._lock:
            # Increment the consecutive failure streak counter.
            self._failure_streak += 1
            # Reset the consecutive success streak counter upon any failure.
            self._success_streak = 0

            # Determine the appropriate increase factor based on whether it's a rate limit error.
            increase_factor = self._rate_limit_error_increase_factor if is_rate_limit_error else self._failure_increase_factor
            # Calculate the new delay by applying the selected increase factor.
            new_delay = self.current_delay_seconds * increase_factor
            # Update the current delay, ensuring it does not exceed the defined maximum delay.
            self.current_delay_seconds = min(self.max_delay_seconds, new_delay)
            # Determine the logging level based on the error type (WARNING for rate limit errors, INFO otherwise).
            log_level = logging.WARNING if is_rate_limit_error else logging.INFO
            # Log a message indicating the delay increase and the reason.
            self._logger.log(log_level, f"Rate limiter delay increased to {self.current_delay_seconds:.3f}s due to failure (rate_limit_error={is_rate_limit_error}).")


In [ ]:
# Defines the 'VertexTTSOrchestrator' class, providing a production-grade solution
# for Text-to-Speech (TTS) synthesis using Google Cloud services.
class VertexTTSOrchestrator:
    """
    Production-grade Text-to-Speech (TTS) orchestrator for Google Cloud.

    This class provides a robust and cost-efficient solution for TTS synthesis,
    incorporating:
    - Comprehensive input validation and error handling.
    - Dynamic discovery and validation of available TTS voices.
    - Intelligent voice selection based on quality preference and cost considerations.
    - Thread-safe operations for shared resources (e.g., cost metrics, cache).
    - True asynchronous API calls for non-blocking I/O.
    - Caching mechanism with Time-To-Live (TTL) for audio responses.
    - Reliability patterns: Circuit Breaker for API resilience, Adaptive Rate Limiter
      for managing request quotas, and retry mechanisms for transient errors.
    - Detailed cost tracking and optimization recommendations.
    - Professional-grade logging for observability and debugging.
    - Concurrency management for batch operations.
    """

    # Defines a class constant for the default Google Cloud region if not specified.
    DEFAULT_REGION = "us-central1"
    # Defines a class constant for the default monthly cost threshold in USD for triggering cost-saving measures.
    DEFAULT_COST_THRESHOLD_USD = 100.0
    # Defines a class constant for the default maximum number of concurrent synthesis requests.
    DEFAULT_MAX_CONCURRENT_REQUESTS = 10
    # Defines a class constant for the default TTL for cached audio items in seconds (e.g., 24 hours).
    DEFAULT_CACHE_TTL_SECONDS = 24 * 60 * 60
    # Defines a class constant for the factor of the cost threshold at which to consider downgrading voice quality.
    COST_THRESHOLD_DOWNGRADE_FACTOR = 0.8
    # Defines a class constant for the maximum number of items in the in-memory cache.
    MAX_CACHE_SIZE = 1000

    # Defines the constructor for the VertexTTSOrchestrator class.
    def __init__(self,
                 project_id: str,
                 region: str = DEFAULT_REGION,
                 enable_caching: bool = True,
                 cache_ttl_seconds: int = DEFAULT_CACHE_TTL_SECONDS,
                 cost_threshold_usd: float = DEFAULT_COST_THRESHOLD_USD,
                 max_concurrent_requests: int = DEFAULT_MAX_CONCURRENT_REQUESTS,
                 enable_circuit_breaker: bool = True,
                 google_application_credentials_path: Optional[str] = None):
        """
        Initializes the VertexTTSOrchestrator with comprehensive validation and setup.

        Args:
            project_id: Google Cloud Project ID. Must be a valid format.
            region: Vertex AI region (e.g., "us-central1"). Must be a supported region.
            enable_caching: If True, enables caching of synthesized audio.
            cache_ttl_seconds: Time-To-Live for cached items in seconds.
            cost_threshold_usd: Monthly cost threshold in USD. If estimated costs approach this,
                                 the orchestrator may switch to more cost-effective voice tiers.
            max_concurrent_requests: Maximum number of concurrent TTS synthesis operations.
            enable_circuit_breaker: If True, enables the circuit breaker pattern for API calls.
            google_application_credentials_path: Optional path to Google Cloud service account JSON key file.
                                                 If None, attempts to use default credentials.

        Raises:
            ConfigurationError: If project_id, region, or other parameters are invalid.
            InitializationError: If client initialization or other setup fails.
        """
        # Obtain a logger instance specific to this VertexTTSOrchestrator class instance.
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        # Log an informational message indicating the start of orchestrator initialization.
        self.logger.info(f"Initializing VertexTTSOrchestrator for project '{project_id}' in region '{region}'.")

        # Validate the format of the provided 'project_id' using an internal helper method.
        if not self._is_valid_project_id(project_id):
            # If 'project_id' is invalid, raise a ConfigurationError.
            raise ConfigurationError(f"Invalid Google Cloud Project ID format: '{project_id}'.")
        # Assign the validated 'project_id' to an instance variable.
        self.project_id = project_id

        # Validate the provided 'region' using an internal helper method.
        if not self._is_valid_vertex_ai_region(region):
            # If 'region' is invalid or unsupported, raise a ConfigurationError.
            raise ConfigurationError(f"Invalid or unsupported Vertex AI region: '{region}'.")
        # Assign the validated 'region' to an instance variable.
        self.region = region

        # Validate that 'cost_threshold_usd' is a positive number (integer or float).
        if not isinstance(cost_threshold_usd, (int, float)) or cost_threshold_usd <= 0:
            # If 'cost_threshold_usd' is invalid, raise a ConfigurationError.
            raise ConfigurationError(f"Cost threshold (cost_threshold_usd) must be a positive number, got {cost_threshold_usd}.")
        # Assign the validated 'cost_threshold_usd' to an instance variable.
        self.cost_threshold_usd = cost_threshold_usd

        # Validate that 'max_concurrent_requests' is a positive integer.
        if not isinstance(max_concurrent_requests, int) or max_concurrent_requests <= 0:
            # If 'max_concurrent_requests' is invalid, raise a ConfigurationError.
            raise ConfigurationError(f"Max concurrent requests must be a positive integer, got {max_concurrent_requests}.")
        # Initialize an asyncio.Semaphore to limit concurrent asynchronous synthesis operations.
        self._async_semaphore = asyncio.Semaphore(max_concurrent_requests)

        # Assign the 'enable_caching' boolean flag to an instance variable.
        self.enable_caching = enable_caching
        # Validate that 'cache_ttl_seconds' is a positive integer.
        if not isinstance(cache_ttl_seconds, int) or cache_ttl_seconds <= 0:
            # If 'cache_ttl_seconds' is invalid, raise a ConfigurationError.
            raise ConfigurationError(f"Cache TTL (cache_ttl_seconds) must be a positive integer, got {cache_ttl_seconds}.")
        # Convert 'cache_ttl_seconds' to a timedelta object and store it as 'self.cache_ttl'.
        self.cache_ttl = timedelta(seconds=cache_ttl_seconds)
        # Initialize an empty dictionary for in-memory cache storage.
        self._cache_storage: Dict[str, Tuple[bytes, datetime]] = {}
        # Initialize a threading.Lock for thread-safe access to the '_cache_storage'.
        self._cache_lock = threading.Lock()
        # Initialize an empty list to maintain cache access order for LRU eviction.
        self._cache_access_order: List[str] = []

        # Initialize an instance of CostMetrics for thread-safe cost tracking.
        self.cost_metrics = CostMetrics()

        # Initialize a CircuitBreaker instance if 'enable_circuit_breaker' is True, otherwise set to None.
        self.circuit_breaker = CircuitBreaker() if enable_circuit_breaker else None
        # Initialize an instance of AdaptiveRateLimiter for managing request rates.
        self.rate_limiter = AdaptiveRateLimiter()

        # Initialize Google Cloud clients using the internal '_init_clients' method.
        self._init_clients(google_application_credentials_path)

        # Initialize voice mappings: 'cost_optimized' and 'premium' tiers, initially empty.
        self.voice_mapping: Dict[str, Dict[str, Dict[str, Any]]] = {"cost_optimized": {}, "premium": {}}
        # Initialize an empty set to store all available voice names discovered from the API.
        self.available_voices: Set[str] = set()
        # Populate 'self.voice_mapping' and 'self.available_voices' by dynamically discovering voices from the TTS API.
        self._initialize_voice_mapping_from_api()

        # Log an informational message indicating successful orchestrator initialization.
        self.logger.info("VertexTTSOrchestrator initialized successfully.")

    # Defines an internal helper method to validate the Google Cloud Project ID format.
    def _is_valid_project_id(self, project_id: str) -> bool:
        """
        Validates the Google Cloud Project ID format.
        Rules: 6-30 chars, lowercase letters, numbers, hyphens. Must start with a letter, cannot end with a hyphen.

        Args:
            project_id: The project ID string to validate.

        Returns:
            bool: True if the project_id format is valid, False otherwise.
        """
        # Check if the provided 'project_id' is actually a string.
        if not isinstance(project_id, str):
            # If not a string, it's an invalid format, so return False.
            return False
        # Define the regular expression pattern for validating Google Cloud project ID constraints.
        pattern = r'^[a-z][a-z0-9-]{4,28}[a-z0-9]$'
        # Perform a full match of the 'project_id' against the defined 'pattern'.
        # Convert the match object (or None) to a boolean value indicating validity.
        return bool(re.fullmatch(pattern, project_id))

    # Defines an internal helper method to validate the Google Cloud region format and check against known Vertex AI regions.
    def _is_valid_vertex_ai_region(self, region: str) -> bool:
        """
        Validates the Google Cloud region format and checks if it's a known Vertex AI supported region.
        Note: This list is not exhaustive and might need updates as Google expands regions.
              For true dynamic validation, an API call to list available regions would be needed,
              but that adds complexity and potential failure points to initialization.

        Args:
            region: The region string to validate (e.g., "us-central1").

        Returns:
            bool: True if the region format is valid and in the known list, False otherwise.
        """
        # Check if the provided 'region' is actually a string.
        if not isinstance(region, str):
            # If not a string, it's an invalid format, so return False.
            return False
        # Define a regular expression pattern for common Google Cloud region name formats.
        pattern = r'^[a-z]+-[a-z0-9]+(?:-[a-z0-9]+)?$'
        # Check if the 'region' string fully matches the defined 'pattern'.
        if not re.fullmatch(pattern, region):
            # If the format doesn't match, return False.
            return False

        # Define a set of known common Vertex AI supported regions for validation.
        known_vertex_ai_regions = {
            'us-central1', 'us-east1', 'us-east4', 'us-west1', 'us-west2', 'us-west4',
            'europe-central2', 'europe-north1', 'europe-southwest1', 'europe-west1',
            'europe-west2', 'europe-west3', 'europe-west4', 'europe-west6', 'europe-west8', 'europe-west9',
            'asia-east1', 'asia-east2', 'asia-northeast1', 'asia-northeast2', 'asia-northeast3',
            'asia-south1', 'asia-southeast1', 'asia-southeast2',
            'australia-southeast1', 'australia-southeast2',
            'northamerica-northeast1', 'northamerica-northeast2',
            'southamerica-east1', 'southamerica-west1',
            'me-central1', 'me-west1', 'africa-south1'
        }
        # Check if the provided 'region' is present in the set of 'known_vertex_ai_regions'.
        return region in known_vertex_ai_regions

    # Defines an internal method to initialize Google Cloud clients with error handling and authentication.
    def _init_clients(self, credentials_path: Optional[str]) -> None:
        """
        Initializes Google Cloud clients (TextToSpeech, Vertex AI, Storage)
        with robust error handling, authentication, and connectivity checks.

        Args:
            credentials_path: Optional path to a service account key file.

        Raises:
            InitializationError: If client initialization or authentication fails.
        """
        # Log an informational message indicating the start of Google Cloud client initialization.
        self.logger.info("Initializing Google Cloud clients...")
        # Begin a try block to handle potential exceptions during client initialization.
        try:
            # Initialize 'creds' to None; it will hold credentials if explicitly found or default ones are used.
            creds = None
            # Check if a 'credentials_path' was provided.
            if credentials_path:
                # Check if the file at 'credentials_path' exists.
                if not os.path.exists(credentials_path):
                    # If the credentials file is not found, raise an InitializationError.
                    raise InitializationError(f"Credentials file not found at: {credentials_path}")
                # Set the 'GOOGLE_APPLICATION_CREDENTIALS' environment variable to the provided path.
                os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
                # Log that credentials from the specified path are being used.
                self.logger.info(f"Using credentials from specified path: {credentials_path}")
            # Else, check if the 'GOOGLE_APPLICATION_CREDENTIALS' environment variable is already set.
            elif os.getenv('GOOGLE_APPLICATION_CREDENTIALS'):
                 # Log that credentials from the environment variable are being used.
                 self.logger.info(f"Using credentials from GOOGLE_APPLICATION_CREDENTIALS environment variable: {os.getenv('GOOGLE_APPLICATION_CREDENTIALS')}")
            # Else, attempt to use default Google Cloud credentials.
            else:
                # Log that an attempt is being made to use default credentials.
                self.logger.info("Attempting to use default Google Cloud credentials.")
                # Begin a try block to handle potential errors when fetching default credentials.
                try:
                    # Attempt to get default credentials and the associated project ID.
                    default_creds, _ = get_default_credentials()
                    # Check if default credentials were successfully obtained.
                    if default_creds is None:
                         # If no default credentials found, raise DefaultCredentialsError.
                         raise DefaultCredentialsError("No default credentials found.")
                    # Assign the obtained default credentials to 'creds'.
                    creds = default_creds
                # Catch DefaultCredentialsError if default credentials are not found or invalid.
                except DefaultCredentialsError as e:
                    # Raise an InitializationError with a detailed message about the authentication failure.
                    raise InitializationError(
                        "Google Cloud authentication failed: Default credentials not found or invalid. "
                        "Ensure you are authenticated (e.g., `gcloud auth application-default login`) "
                        "or provide a service account key file."
                    ) from e

            # Initialize the Text-to-Speech client.
            # Pass 'creds' only if they were explicitly obtained via default mechanism and no path/env var was set.
            # Client libraries typically pick up GOOGLE_APPLICATION_CREDENTIALS automatically if set.
            self.tts_client = texttospeech.TextToSpeechClient(credentials=creds if creds and not credentials_path and not os.getenv('GOOGLE_APPLICATION_CREDENTIALS') else None)
            # Perform a connectivity test for the initialized TTS client.
            self._test_tts_client_connectivity()

            # Initialize the Vertex AI SDK.
            # Pass 'creds' under the same conditions as for the TTS client.
            vertexai.init(project=self.project_id, location=self.region, credentials=creds if creds and not credentials_path and not os.getenv('GOOGLE_APPLICATION_CREDENTIALS') else None)
            # Log successful initialization of the Vertex AI SDK.
            self.logger.info(f"Vertex AI SDK initialized for project '{self.project_id}' in location '{self.region}'.")

            # Note: Storage client initialization for GCS caching is commented out as in-memory cache is used.
            # if self.enable_caching:
            #     self.storage_client = storage.Client(project=self.project_id, credentials=creds)
            #     self._test_storage_client_connectivity()

            # Log successful initialization of all Google Cloud clients.
            self.logger.info("All Google Cloud clients initialized successfully.")

        # Catch GoogleCloudError specifically, which can occur during client interactions.
        except GoogleCloudError as e:
            # Log the Google Cloud error that occurred during client initialization.
            self.logger.error(f"A Google Cloud error occurred during client initialization: {e}", exc_info=True)
            # Raise an InitializationError, wrapping the original GoogleCloudError.
            raise InitializationError(f"Failed to initialize Google Cloud clients due to GoogleCloudError: {e}") from e
        # Catch any other unexpected exceptions during client initialization.
        except Exception as e:
            # Log the unexpected error.
            self.logger.error(f"An unexpected error occurred during client initialization: {e}", exc_info=True)
            # Raise an InitializationError, wrapping the original unexpected error.
            raise InitializationError(f"Unexpected error during client initialization: {e}") from e

    # Defines an internal method to test connectivity to the Google Cloud Text-to-Speech API.
    def _test_tts_client_connectivity(self) -> None:
        """
        Tests connectivity to the Google Cloud Text-to-Speech API by listing available voices.

        Raises:
            InitializationError: If the connectivity test fails.
        """
        # Check if the TTS client has been initialized before attempting a connectivity test.
        if not self.tts_client:
             # If the TTS client is not initialized, raise an InitializationError.
             raise InitializationError("TTS client is not initialized before testing connectivity.")
        # Log an informational message indicating the start of the TTS client connectivity test.
        self.logger.info("Testing Text-to-Speech client connectivity...")
        # Begin a try block to handle potential exceptions during the API call.
        try:
            # Make a lightweight API call (list_voices) to verify connectivity and authentication.
            voices_response = self.tts_client.list_voices()
            # Check if the API call returned any voices.
            if not voices_response.voices:
                # Log a warning if no voices were returned, though the call itself was successful.
                self.logger.warning("TTS API connectivity test: list_voices returned no voices, but call was successful.")
            # Handle the case where voices were successfully listed.
            else:
                # Log an informational message confirming successful TTS client connectivity and the number of voices found.
                self.logger.info(f"TTS client connectivity verified: Successfully listed {len(voices_response.voices)} available voices.")
        # Catch GoogleCloudError specifically, which can occur during the API call.
        except GoogleCloudError as e:
            # Log the error that occurred during the connectivity test.
            self.logger.error(f"TTS client connectivity test failed: {e}", exc_info=True)
            # Raise an InitializationError, wrapping the original GoogleCloudError.
            raise InitializationError(f"TTS client connectivity test failed: {e}") from e
        # Catch any other unexpected exceptions during the connectivity test.
        except Exception as e:
            # Log the unexpected error.
            self.logger.error(f"Unexpected error during TTS client connectivity test: {e}", exc_info=True)
            # Raise an InitializationError, wrapping the original unexpected error.
            raise InitializationError(f"Unexpected error during TTS client connectivity test: {e}") from e

    # Defines an internal method to dynamically discover and categorize available TTS voices from the API.
    def _initialize_voice_mapping_from_api(self) -> None:
        """
        Dynamically discovers available voices from the Google Cloud TTS API and
        categorizes them into 'cost_optimized' (Standard) and 'premium' (Neural2, WaveNet) tiers.
        Populates `self.voice_mapping` and `self.available_voices`.

        Falls back to a minimal hardcoded mapping if API discovery fails.
        """
        # Check if the TTS client has been initialized before attempting voice discovery.
        if not self.tts_client:
            # Log an error if the TTS client is not available.
            self.logger.error("TTS client not initialized. Cannot perform dynamic voice discovery.")
            # Initialize voice mapping using a fallback mechanism.
            self._initialize_fallback_voice_mapping("TTS client not available for dynamic discovery.")
            # Return early as dynamic discovery cannot proceed.
            return

        # Log an informational message indicating the start of voice mapping initialization from the API.
        self.logger.info("Initializing voice mapping from Google Cloud TTS API...")
        # Begin a try block to handle potential exceptions during API interaction.
        try:
            # Retrieve the list of all available voices from the Google Cloud TTS API.
            voices_response = self.tts_client.list_voices()
            # Check if the API response contains any voices.
            if not voices_response.voices:
                # Log a warning if no voices were returned by the API.
                self.logger.warning("No voices returned from TTS API during dynamic discovery.")
                # Initialize voice mapping using a fallback mechanism.
                self._initialize_fallback_voice_mapping("API returned no voices.")
                # Return early as there are no voices to process.
                return

            # Initialize temporary dictionaries to build the voice mapping for cost-optimized voices.
            cost_optimized_map: Dict[str, Dict[str, Any]] = {}
            # Initialize temporary dictionaries to build the voice mapping for premium voices.
            premium_map: Dict[str, Dict[str, Any]] = {}

            # Iterate over each 'voice' object in the 'voices_response.voices' list.
            for voice in voices_response.voices:
                # Check if the current 'voice' has any associated language codes.
                if not voice.language_codes:
                    # Log a debug message and skip this voice if it has no language codes.
                    self.logger.debug(f"Skipping voice '{voice.name}' as it has no language codes specified.")
                    # Continue to the next voice in the list.
                    continue

                # Get the primary language code for the current voice (typically the first one listed).
                primary_language_code = voice.language_codes[0]
                # Determine the VoiceQuality tier (e.g., STANDARD, NEURAL2, WAVENET) from the voice name.
                quality = VoiceQuality.from_voice_name(voice.name)

                # Prepare a dictionary containing detailed information about the current voice.
                voice_info = {
                    "name": voice.name,
                    "gender": voice.ssml_gender,
                    "quality_tier_enum": quality,
                    "natural_sample_rate_hertz": voice.natural_sample_rate_hertz,
                    "all_language_codes": list(voice.language_codes)
                }
                # Add the name of the current voice to the set of all available voice names for quick validation.
                self.available_voices.add(voice.name)

                # Categorize and store the voice based on its quality tier.
                # Check if the voice quality is STANDARD.
                if quality == VoiceQuality.STANDARD:
                    # If this language code is not yet in 'cost_optimized_map', add this voice.
                    # This prefers the first standard voice encountered for a given language.
                    if primary_language_code not in cost_optimized_map:
                         # Add the 'voice_info' to the 'cost_optimized_map' for its primary language code.
                         cost_optimized_map[primary_language_code] = voice_info
                # Handle premium voices (Neural2 or WaveNet).
                else:
                    # Get the currently stored premium voice for this language code, if any.
                    current_premium_voice = premium_map.get(primary_language_code)
                    # Prefer Neural2 voices over WaveNet if both exist for a language, or add if no premium voice exists yet.
                    if not current_premium_voice or \
                       (quality == VoiceQuality.NEURAL2 and current_premium_voice["quality_tier_enum"] == VoiceQuality.WAVENET):
                        # Add or update the 'voice_info' in the 'premium_map' for its primary language code.
                        premium_map[primary_language_code] = voice_info

            # Assign the populated 'cost_optimized_map' to the instance's voice mapping.
            self.voice_mapping["cost_optimized"] = cost_optimized_map
            # Assign the populated 'premium_map' to the instance's voice mapping.
            self.voice_mapping["premium"] = premium_map

            # Ensure that essential fallback voices (e.g., for 'en-US') are present in the mapping.
            self._ensure_fallback_voices_present()

            # Log a summary of the successful voice mapping initialization.
            self.logger.info(f"Voice mapping initialized successfully: "
                             f"{len(self.available_voices)} total voices discovered. "
                             f"{len(cost_optimized_map)} languages in cost-optimized tier, "
                             f"{len(premium_map)} languages in premium tier.")

        # Catch GoogleCloudError specifically, which can occur during API interactions.
        except GoogleCloudError as e:
            # Log the error and indicate that a fallback mapping will be used.
            self.logger.error(f"GoogleCloudError during dynamic voice discovery: {e}. Using fallback mapping.", exc_info=True)
            # Initialize voice mapping using the fallback mechanism.
            self._initialize_fallback_voice_mapping(f"GoogleCloudError: {e}")
        # Catch any other unexpected exceptions during voice discovery.
        except Exception as e:
            # Log the unexpected error and indicate that a fallback mapping will be used.
            self.logger.error(f"Unexpected error during dynamic voice discovery: {e}. Using fallback mapping.", exc_info=True)
            # Initialize voice mapping using the fallback mechanism.
            self._initialize_fallback_voice_mapping(f"Unexpected error: {e}")

    # Defines an internal method to initialize a minimal, hardcoded voice mapping as a fallback.
    def _initialize_fallback_voice_mapping(self, reason: str) -> None:
        """
        Initializes a minimal, hardcoded voice mapping as a fallback
        if dynamic discovery from the API fails.

        Args:
            reason: The reason why fallback mapping is being used.
        """
        # Log a warning message indicating that fallback voice mapping is being used and the reason.
        self.logger.warning(f"Using fallback voice mapping due to: {reason}")
        # Define a basic, hardcoded voice mapping with common 'en-US' voices.
        self.voice_mapping = {
            "cost_optimized": {
                "en-US": {
                    "name": "en-US-Standard-C", "gender": texttospeech.SsmlVoiceGender.FEMALE,
                    "quality_tier_enum": VoiceQuality.STANDARD, "natural_sample_rate_hertz": 24000,
                    "all_language_codes": ["en-US"]
                }
            },
            "premium": {
                "en-US": {
                    "name": "en-US-Neural2-C", "gender": texttospeech.SsmlVoiceGender.FEMALE,
                    "quality_tier_enum": VoiceQuality.NEURAL2, "natural_sample_rate_hertz": 24000,
                    "all_language_codes": ["en-US"]
                }
            }
        }
        # Clear any existing entries in 'self.available_voices'.
        self.available_voices.clear()
        # Iterate through the tiers in the fallback 'self.voice_mapping'.
        for tier_map in self.voice_mapping.values():
            # Iterate through the voice data in each language within the tier.
            for lang_voices_data in tier_map.values(): # Corrected variable name
                # Add the voice name to the 'self.available_voices' set.
                self.available_voices.add(lang_voices_data["name"]) # Corrected variable name
        # Log an informational message about the initialized fallback voice mapping.
        self.logger.info(f"Fallback voice mapping initialized with {len(self.available_voices)} voices.")

    # Defines an internal method to ensure essential fallback voices (e.g., for 'en-US') are present.
    def _ensure_fallback_voices_present(self) -> None:
        """
        Checks if essential fallback voices (e.g., for 'en-US') are present in the
        dynamically generated voice mapping. Logs a warning if not found.
        This helps ensure basic functionality even if specific regional voices are missing.
        """
        # Define the default fallback language code (e.g., 'en-US').
        default_fallback_lang = "en-US"
        # Check if the 'default_fallback_lang' is present in the 'cost_optimized' tier of the voice mapping.
        if default_fallback_lang not in self.voice_mapping["cost_optimized"]:
            # Log a warning if the default fallback language is not found in the cost-optimized tier.
            self.logger.warning(f"Default fallback language '{default_fallback_lang}' not found in 'cost_optimized' voice tier after dynamic discovery.")
            # Iterate through existing languages in the cost-optimized tier to find a suitable 'en-' voice.
            for lang, voice_data in self.voice_mapping["cost_optimized"].items():
                # Check if the language code starts with "en-".
                if lang.startswith("en-"):
                    # Assign this 'en-' voice data as the fallback for 'default_fallback_lang'.
                    self.voice_mapping["cost_optimized"][default_fallback_lang] = voice_data
                    # Log that an alternative 'en-' voice has been assigned as fallback.
                    self.logger.info(f"Assigned '{voice_data['name']}' as fallback for '{default_fallback_lang}' in cost_optimized tier.")
                    # Break the loop as a suitable fallback has been found and assigned.
                    break

        # Check if the 'default_fallback_lang' is present in the 'premium' tier of the voice mapping.
        if default_fallback_lang not in self.voice_mapping["premium"]:
            # Log a warning if the default fallback language is not found in the premium tier.
            self.logger.warning(f"Default fallback language '{default_fallback_lang}' not found in 'premium' voice tier after dynamic discovery.")
            # Iterate through existing languages in the premium tier to find a suitable 'en-' voice.
            for lang, voice_data in self.voice_mapping["premium"].items():
                # Check if the language code starts with "en-".
                if lang.startswith("en-"):
                    # Assign this 'en-' voice data as the fallback for 'default_fallback_lang'.
                    self.voice_mapping["premium"][default_fallback_lang] = voice_data
                    # Log that an alternative 'en-' voice has been assigned as fallback.
                    self.logger.info(f"Assigned '{voice_data['name']}' as fallback for '{default_fallback_lang}' in premium tier.")
                    # Break the loop as a suitable fallback has been found and assigned.
                    break

    # Defines an internal helper method to get voice configuration from a tier map with simple language fallback.
    def _get_voice_config_from_mapping(self, language_code: str, quality_tier_map: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """
        Helper to get voice config from a tier map, with simple language fallback (e.g., 'en-GB' to 'en-US').
        """
        # Attempt to get the voice configuration directly using the provided 'language_code'.
        voice_config = quality_tier_map.get(language_code)
        # Check if 'voice_config' was not found and if 'language_code' contains a region subtag (e.g., 'en-GB').
        if not voice_config and '-' in language_code:
            # Extract the generic language part (e.g., 'en' from 'en-GB').
            generic_lang = language_code.split('-')[0]
            # Iterate through the language codes and voice configurations in the 'quality_tier_map'.
            for lc, vc in quality_tier_map.items():
                # Check if the current language code 'lc' matches the generic language or starts with the generic language prefix.
                if lc == generic_lang or lc.startswith(f"{generic_lang}-"):
                    # Log a debug message indicating the fallback to a more generic language variant.
                    self.logger.debug(f"Language '{language_code}' not found in tier, falling back to '{lc}'.")
                    # Return the voice configuration 'vc' of the first suitable generic match.
                    return vc
        # Return the 'voice_config' (which could be None if no direct match or fallback was found).
        return voice_config

    # Defines an internal method to calculate the estimated cost for a TTSRequest.
    def _calculate_estimated_cost(self, tts_request: TTSRequest, actual_quality_tier: VoiceQuality) -> float:
        """
        Calculates the estimated cost for a given TTSRequest based on its character count
        and the actual voice quality tier to be used, considering Google Cloud free tiers.

        Args:
            tts_request: The TTSRequest object containing text and other details.
            actual_quality_tier: The VoiceQuality enum member that will actually be used for synthesis.

        Returns:
            float: The estimated cost in USD for this synthesis request.
        """
        # Get the billable character count from the 'tts_request' (handles SSML stripping).
        char_count = tts_request.get_billable_character_count()
        # If the character count is zero, there is no cost.
        if char_count == 0:
            # Return 0.0 as the estimated cost.
            return 0.0

        # Get a snapshot of the current cost metrics for estimation.
        cost_metrics_snapshot = self.cost_metrics.get_current_metrics_snapshot()
        # Get the free tier character limit for the 'actual_quality_tier'.
        free_tier_limit = actual_quality_tier.get_free_tier_character_limit()
        # Get the cost per million characters for the 'actual_quality_tier'.
        cost_per_million_chars = actual_quality_tier.get_cost_per_million_characters()

        # Initialize 'current_tier_usage' to 0.
        current_tier_usage = 0
        # Check if the 'actual_quality_tier' is STANDARD.
        if actual_quality_tier == VoiceQuality.STANDARD:
            # Assign the standard voice usage from the metrics snapshot.
            current_tier_usage = cost_metrics_snapshot['standard_voice_usage']
        # Handle premium quality tiers (WaveNet, Neural2).
        else:
            # Assign the premium voice usage from the metrics snapshot.
            current_tier_usage = cost_metrics_snapshot['premium_voice_usage']

        # Calculate the remaining characters in the free tier for this quality tier.
        remaining_free_tier_chars = max(0, free_tier_limit - current_tier_usage)
        # Determine how many characters of the current request fall into the free tier.
        chars_in_free_tier_for_this_request = min(char_count, remaining_free_tier_chars)

        # Calculate how many characters of the current request will be paid (billed).
        paid_chars_in_this_request = char_count - chars_in_free_tier_for_this_request

        # Initialize 'estimated_cost' to 0.0.
        estimated_cost = 0.0
        # Check if there are any characters to be paid for in this request.
        if paid_chars_in_this_request > 0:
            # Calculate the estimated cost based on the number of paid characters and cost per million.
            estimated_cost = (paid_chars_in_this_request * cost_per_million_chars) / 1_000_000.0

        # Log a debug message detailing the cost estimation parameters and result.
        self.logger.debug(f"Request {tts_request.request_id}: Cost estimation: "
                          f"{char_count} chars, Tier: {actual_quality_tier.value}, "
                          f"Current Tier Usage: {current_tier_usage}, Free Limit: {free_tier_limit}, "
                          f"Paid Chars in this Req: {paid_chars_in_this_request}, Est. Cost: ${estimated_cost:.6f}")
        # Return the calculated 'estimated_cost'.
        return estimated_cost

    # Defines an internal method to generate a deterministic cache key for a TTSRequest using SHA-256.
    def _generate_cache_key(self, tts_request: TTSRequest, final_voice_name: str) -> str:
        """
        Generates a deterministic cache key for a TTSRequest using SHA-256.
        The key incorporates all parameters that affect the synthesized audio output,
        including the final selected voice name. Unicode normalization is applied to text.

        Args:
            tts_request: The TTSRequest object.
            final_voice_name: The actual voice name that will be used for synthesis.

        Returns:
            str: A SHA-256 hash string representing the cache key.
        """
        # Initialize 'normalized_text' with the raw text from the request.
        normalized_text: str
        # Begin a try block to handle potential TypeErrors during Unicode normalization.
        try:
            # Normalize the Unicode text from 'tts_request.text' to NFKC form for canonical representation.
            normalized_text = unicodedata.normalize('NFKC', tts_request.text)
        # Catch TypeError, which might occur if 'tts_request.text' is not a string (though validated).
        except TypeError as e:
            # Log an error if Unicode normalization fails, and use the raw text as a fallback.
            self.logger.error(f"Error normalizing text for cache key (request {tts_request.request_id}): {e}. Using raw text.")
            # Assign the raw text to 'normalized_text'.
            normalized_text = tts_request.text

        # Construct a composite string from all relevant parameters that affect audio output for consistent hashing.
        cache_data_string = (
            f"text:{normalized_text}|"
            f"lang:{tts_request.language_code}|"
            f"voice:{final_voice_name}|"
            f"ssml:{tts_request.ssml_enabled}|"
            f"rate:{tts_request.speaking_rate:.2f}|"
            f"pitch:{tts_request.pitch:.2f}|"
            f"encoding:{tts_request.audio_encoding.value}"
        )

        # Encode the composite 'cache_data_string' to UTF-8 bytes before hashing.
        encoded_data = cache_data_string.encode('utf-8')
        # Generate a SHA-256 hash of the encoded data.
        sha256_hash = hashlib.sha256(encoded_data)
        # Return the hexadecimal representation of the SHA-256 hash digest.
        return sha256_hash.hexdigest()

    # Defines an internal method for intelligent voice selection based on request parameters and cost constraints.
    def _intelligent_voice_selection(self, tts_request: TTSRequest) -> Dict[str, Any]:
        """
        Intelligently selects the most appropriate voice configuration for a given TTSRequest.
        Considers requested voice name, quality preference, language, cost constraints,
        and availability from the dynamically discovered voice mapping.

        Args:
            tts_request: The TTSRequest object.

        Returns:
            Dict[str, Any]: A dictionary containing the selected voice configuration,
                            including 'name', 'gender', 'quality_tier_enum', etc.

        Raises:
            VoiceSelectionError: If a suitable voice cannot be found or if the requested
                                 voice is invalid or unavailable.
        """
        # Log a debug message indicating the start of the intelligent voice selection process.
        self.logger.debug(f"Request {tts_request.request_id}: Starting intelligent voice selection for lang '{tts_request.language_code}', quality '{tts_request.quality_tier.value}'.")

        # Initialize 'selected_voice_config' to None.
        selected_voice_config: Optional[Dict[str, Any]] = None

        # --- Step 1: Handle explicitly requested voice_name ---
        # Check if a specific 'voice_name' was provided in the 'tts_request'.
        if tts_request.voice_name:
            # Validate if the requested 'voice_name' is in the set of available voices.
            if tts_request.voice_name not in self.available_voices:
                # If the voice is not available, raise a VoiceSelectionError.
                raise VoiceSelectionError(f"Request {tts_request.request_id}: Requested voice '{tts_request.voice_name}' is not available or invalid.")
            # Iterate through the voice mapping tiers ('cost_optimized', 'premium') to find the requested voice.
            for tier in ["cost_optimized", "premium"]:
                # Iterate through language-to-voice_config mappings within the current tier.
                for lang_map_voice_config in self.voice_mapping[tier].values(): # Corrected variable name
                    # Check if the 'name' in the current 'lang_map_voice_config' matches the requested 'voice_name'.
                    if lang_map_voice_config["name"] == tts_request.voice_name:
                        # If a match is found, assign it to 'selected_voice_config'.
                        selected_voice_config = lang_map_voice_config
                        # Break the inner loop as the voice has been found.
                        break
                # Check if 'selected_voice_config' has been found in the current tier.
                if selected_voice_config:
                    # Break the outer loop as the voice has been found.
                    break
            # This check is a safeguard; if voice_name is in available_voices, it should be found in voice_mapping.
            if not selected_voice_config:
                 # If configuration for an available voice is not found, raise a VoiceSelectionError indicating a mapping issue.
                 raise VoiceSelectionError(f"Request {tts_request.request_id}: Could not find configuration for available voice '{tts_request.voice_name}'. Mapping issue.")

            # --- Cost constraint check for explicitly requested premium voice ---
            # Get a snapshot of the current cost metrics.
            current_cost_snapshot = self.cost_metrics.get_current_metrics_snapshot()
            # Get the current estimated monthly cost from the snapshot.
            current_monthly_cost = current_cost_snapshot['estimated_monthly_cost']
            # Check if the selected voice is premium and if the cost threshold is being approached or exceeded.
            if selected_voice_config["quality_tier_enum"] != VoiceQuality.STANDARD and \
               current_monthly_cost > self.cost_threshold_usd * self.COST_THRESHOLD_DOWNGRADE_FACTOR:
                # Log a warning that a premium voice was requested but cost constraints suggest downgrading.
                self.logger.warning(f"Request {tts_request.request_id}: Requested premium voice '{tts_request.voice_name}' but cost threshold exceeded. "
                                    f"Attempting to downgrade to standard voice for language '{tts_request.language_code}'.")
                # Do not return here; fall through to standard voice selection logic below.
                # Reset selected_voice_config to None so that the logic below re-evaluates for a standard voice.
                selected_voice_config = None
            # If no cost constraint issue for the explicitly requested voice.
            else:
                # Log that the explicitly requested voice will be used.
                self.logger.info(f"Request {tts_request.request_id}: Using explicitly requested voice '{tts_request.voice_name}'.")
                # Return the configuration of the explicitly requested voice.
                return selected_voice_config

        # --- Step 2: Cost Threshold Check for default quality preference (if no explicit voice was chosen or if downgraded) ---
        # Get the initially preferred quality tier from the request.
        effective_quality_tier = tts_request.quality_tier
        # Check if 'selected_voice_config' is still None (meaning no explicit valid voice was chosen or it was nullified by cost check).
        if selected_voice_config is None: # This block executes if no explicit voice is used or if it was overridden by cost
            # Get a snapshot of the current cost metrics if not already fetched.
            # current_cost_snapshot = self.cost_metrics.get_current_metrics_snapshot() # Already fetched if explicit voice was premium
            # current_monthly_cost = current_cost_snapshot['estimated_monthly_cost'] # Already fetched
            # Re-fetch if not already available (e.g. if tts_request.voice_name was None)
            if not tts_request.voice_name: # Only re-fetch if not done in the explicit voice section
                current_cost_snapshot = self.cost_metrics.get_current_metrics_snapshot()
                current_monthly_cost = current_cost_snapshot['estimated_monthly_cost']

            # Check if the preferred quality is premium and if the cost threshold suggests downgrading.
            if tts_request.quality_tier != VoiceQuality.STANDARD and \
               current_monthly_cost > self.cost_threshold_usd * self.COST_THRESHOLD_DOWNGRADE_FACTOR:
                # Log a warning about switching to STANDARD quality due to cost constraints.
                self.logger.warning(f"Request {tts_request.request_id}: Approaching/exceeded cost threshold (${self.cost_threshold_usd:.2f}). "
                                    f"Switching preferred quality from '{tts_request.quality_tier.value}' to '{VoiceQuality.STANDARD.value}' "
                                    f"for language '{tts_request.language_code}'.")
                # Set the 'effective_quality_tier' to STANDARD.
                effective_quality_tier = VoiceQuality.STANDARD

        # --- Step 3: Select voice based on (potentially adjusted) quality tier and language ---
        # Determine the target tier map name based on 'effective_quality_tier'.
        target_tier_map_name = "cost_optimized" if effective_quality_tier == VoiceQuality.STANDARD else "premium"
        # Get the actual dictionary for the target tier from 'self.voice_mapping'.
        target_tier_map = self.voice_mapping[target_tier_map_name]
        # Attempt to get the voice configuration using the 'tts_request.language_code' and the 'target_tier_map'.
        selected_voice_config = self._get_voice_config_from_mapping(tts_request.language_code, target_tier_map)

        # --- Step 4: Fallback logic if specific language/tier combination is not found ---
        # Check if 'selected_voice_config' is still None after the initial attempt.
        if not selected_voice_config:
            # Log a warning that a voice was not found for the specific language and tier.
            self.logger.warning(f"Request {tts_request.request_id}: Voice not found for language '{tts_request.language_code}' in '{target_tier_map_name}' tier.")
            # --- Fallback a: If premium was desired, try standard for the same language. ---
            # Check if the original target was 'premium'.
            if target_tier_map_name == "premium":
                # Log an attempt to fall back to the 'cost_optimized' tier for the same language.
                self.logger.debug(f"Request {tts_request.request_id}: Attempting fallback to 'cost_optimized' tier for language '{tts_request.language_code}'.")
                # Attempt to get voice config from the 'cost_optimized' tier.
                selected_voice_config = self._get_voice_config_from_mapping(tts_request.language_code, self.voice_mapping["cost_optimized"])
                # If a voice is found in the fallback tier, update 'effective_quality_tier'.
                if selected_voice_config:
                    effective_quality_tier = VoiceQuality.STANDARD

            # --- Fallback b: If still not found, try a default/generic language (e.g., 'en-US'). ---
            # Check if 'selected_voice_config' is still None.
            if not selected_voice_config:
                # Define the default fallback language (e.g., 'en-US').
                default_fallback_lang = "en-US"
                # Log an attempt to fall back to the 'default_fallback_lang' in the 'target_tier_map_name' (original or standard if premium failed).
                self.logger.debug(f"Request {tts_request.request_id}: Attempting fallback to language '{default_fallback_lang}' in '{target_tier_map_name}' tier.")
                # Attempt to get voice config for the 'default_fallback_lang' in the 'target_tier_map'.
                selected_voice_config = self._get_voice_config_from_mapping(default_fallback_lang, target_tier_map)
                # If still not found and the target was premium, try 'default_fallback_lang' in 'cost_optimized' tier.
                if not selected_voice_config and target_tier_map_name == "premium":
                    # Log an attempt to fall back to 'default_fallback_lang' in the 'cost_optimized' tier.
                    self.logger.debug(f"Request {tts_request.request_id}: Attempting fallback to language '{default_fallback_lang}' in 'cost_optimized' tier.")
                    # Attempt to get voice config for 'default_fallback_lang' from 'cost_optimized' tier.
                    selected_voice_config = self._get_voice_config_from_mapping(default_fallback_lang, self.voice_mapping["cost_optimized"])
                    # If a voice is found, update 'effective_quality_tier'.
                    if selected_voice_config:
                        effective_quality_tier = VoiceQuality.STANDARD

        # --- Step 5: Final check and return ---
        # Check if 'selected_voice_config' is still None after all selection and fallback attempts.
        if not selected_voice_config:
            # If no suitable voice could be found, raise a VoiceSelectionError.
            raise VoiceSelectionError(f"Request {tts_request.request_id}: Unable to find any suitable voice for language '{tts_request.language_code}' "
                                      f"with quality preference '{tts_request.quality_tier.value}' after all fallbacks.")

        # Ensure the 'selected_voice_config' reflects the 'effective_quality_tier' if it was changed by fallback logic.
        selected_voice_config["quality_tier_enum"] = effective_quality_tier
        # Log the details of the intelligently selected voice.
        self.logger.info(f"Request {tts_request.request_id}: Intelligently selected voice: '{selected_voice_config['name']}' "
                         f"(Quality: {selected_voice_config['quality_tier_enum'].value}, Gender: {selected_voice_config['gender']}).")
        # Return the final 'selected_voice_config'.
        return selected_voice_config

    # Defines an asynchronous method to retrieve an item from the cache if it exists and is not expired.
    async def _get_from_cache(self, cache_key: str) -> Optional[bytes]:
        """
        Retrieves an item from the cache if it exists and is not expired.
        Thread-safe.

        Args:
            cache_key: The key for the cached item.

        Returns:
            The cached audio content (bytes) if found and valid, else None.
        """
        # Check if caching is disabled for the orchestrator.
        if not self.enable_caching:
            # If caching is disabled, return None immediately.
            return None

        # Acquire the cache lock to ensure thread-safe access to the cache storage and access order list.
        with self._cache_lock:
            # Attempt to retrieve the item associated with 'cache_key' from '_cache_storage'.
            cached_item = self._cache_storage.get(cache_key)
            # Check if the 'cached_item' was found.
            if cached_item:
                # Unpack the cached item into 'audio_content' and 'timestamp_utc'.
                audio_content, timestamp_utc = cached_item
                # Check if the cache item has expired by comparing its timestamp with the current time and TTL.
                if datetime.now(timezone.utc) - timestamp_utc < self.cache_ttl:
                    # Log a debug message indicating a cache hit.
                    self.logger.debug(f"Cache hit for key: {cache_key}")
                    # Attempt to remove the 'cache_key' from '_cache_access_order' if it exists (it should).
                    if cache_key in self._cache_access_order:
                        # Remove the key from its current position in the access order list.
                        self._cache_access_order.remove(cache_key)
                    # Append the 'cache_key' to the end of '_cache_access_order' to mark it as recently used (LRU).
                    self._cache_access_order.append(cache_key)
                    # Return the 'audio_content' from the cache.
                    return audio_content
                # Handle the case where the cache item has expired.
                else:
                    # Log a debug message indicating a stale cache item.
                    self.logger.debug(f"Cache stale for key: {cache_key}. Removing.")
                    # Delete the expired item from '_cache_storage'.
                    del self._cache_storage[cache_key]
                    # Attempt to remove the 'cache_key' from '_cache_access_order' if it exists.
                    if cache_key in self._cache_access_order:
                         # Remove the key from the access order list.
                         self._cache_access_order.remove(cache_key)
        # If the item was not found in cache or was expired and removed, return None.
        return None

    # Defines an asynchronous method to put an item into the cache with a creation timestamp.
    async def _put_in_cache(self, cache_key: str, audio_content: bytes) -> None:
        """
        Puts an item into the cache with a creation timestamp.
        Handles cache eviction (simple LRU) if MAX_CACHE_SIZE is exceeded.
        Thread-safe.

        Args:
            cache_key: The key for the item.
            audio_content: The audio content (bytes) to cache.
        """
        # Check if caching is disabled for the orchestrator.
        if not self.enable_caching:
            # If caching is disabled, return immediately without performing any cache operation.
            return

        # Acquire the cache lock to ensure thread-safe access to the cache storage and access order list.
        with self._cache_lock:
            # Check if the cache is full and the current 'cache_key' is not already in the cache (i.e., it's a new item).
            if len(self._cache_storage) >= self.MAX_CACHE_SIZE and cache_key not in self._cache_storage:
                # Check if there are items in the access order list to evict.
                if self._cache_access_order:
                    # Remove the least recently used key (at the beginning of the list).
                    lru_key = self._cache_access_order.pop(0)
                    # Check if the 'lru_key' is present in '_cache_storage' before attempting to delete.
                    if lru_key in self._cache_storage:
                        # Delete the LRU item from '_cache_storage'.
                        del self._cache_storage[lru_key]
                        # Log an informational message about the LRU eviction.
                        self.logger.info(f"Cache full. Evicted LRU item with key: {lru_key}")

            # Add or update the item in '_cache_storage' with the 'audio_content' and current UTC timestamp.
            self._cache_storage[cache_key] = (audio_content, datetime.now(timezone.utc))
            # Update the access order for LRU: remove if exists, then append to mark as recently used.
            # Check if 'cache_key' is already in '_cache_access_order'.
            if cache_key in self._cache_access_order:
                # Remove the key from its current position.
                self._cache_access_order.remove(cache_key)
            # Append 'cache_key' to the end of the list.
            self._cache_access_order.append(cache_key)
            # Log a debug message indicating that the item has been cached.
            self.logger.debug(f"Cached item with key: {cache_key}")

    # Defines the core asynchronous method for performing Text-to-Speech synthesis for a single request.
    async def synthesize_speech(self, tts_request: TTSRequest) -> Tuple[Optional[bytes], Dict[str, Any]]:
        """
        Performs Text-to-Speech synthesis for a single TTSRequest.
        This is the core method, incorporating caching, intelligent voice selection,
        cost calculation, API call with reliability patterns (circuit breaker, rate limiter, retry),
        and metrics updates. This method is truly asynchronous.

        Args:
            tts_request: The TTSRequest object to process.

        Returns:
            A tuple containing:
            - Optional[bytes]: The synthesized audio content as bytes if successful, else None.
            - Dict[str, Any]: Metadata about the synthesis operation, including source (cache/API),
                              voice used, cost, character count, and any errors.
        """
        # Record the start time of the operation for performance measurement.
        op_start_time = time.perf_counter()
        # Log an informational message indicating the start of synthesis for the request.
        self.logger.info(f"Request {tts_request.request_id}: Starting synthesis for text (first 30 chars): '{tts_request.text[:30]}...'")

        # Initialize a dictionary to store metadata about the synthesis operation.
        metadata: Dict[str, Any] = {
            "request_id": tts_request.request_id,
            "source": "unknown",
            "error": None,
            "timestamp_utc": datetime.now(timezone.utc).isoformat()
        }

        # Begin a try block to handle various exceptions that may occur during the synthesis process.
        try:
            # --- Step 1: Validate TTSRequest (already done in __post_init__, but can be a safeguard) ---
            # (Validation is assumed to be handled by TTSRequest.__post_init__ upon object creation)

            # --- Step 2: Intelligent Voice Selection ---
            # Select the actual voice configuration (name, gender, quality tier) to be used.
            selected_voice_config = self._intelligent_voice_selection(tts_request)
            # Extract the final voice name from the selected configuration.
            final_voice_name = selected_voice_config["name"]
            # Extract the actual VoiceQuality enum member to be used.
            actual_quality_tier = selected_voice_config["quality_tier_enum"]
            # Extract the SSML gender from the selected configuration.
            ssml_gender = selected_voice_config["gender"]

            # Update the 'metadata' dictionary with details of the selected voice.
            metadata.update({
                "selected_voice_name": final_voice_name,
                "selected_quality_tier": actual_quality_tier.value,
                "selected_gender": ssml_gender.name
            })

            # --- Step 3: Generate Cache Key ---
            # Generate a cache key based on the final selected parameters and request details.
            tts_request.cache_key = self._generate_cache_key(tts_request, final_voice_name)
            # Add the generated cache key to the 'metadata'.
            metadata["cache_key"] = tts_request.cache_key

            # --- Step 4: Check Cache ---
            # Check if caching is enabled for the orchestrator.
            if self.enable_caching:
                # Attempt to retrieve audio content from the cache using the generated cache key.
                cached_audio = await self._get_from_cache(tts_request.cache_key)
                # Check if audio content was found in the cache.
                if cached_audio:
                    # Update cache metrics to record a cache hit.
                    self.cost_metrics.update_cache_metrics(cache_hit=True)
                    # Set the 'source' in metadata to "cache".
                    metadata["source"] = "cache"
                    # Calculate the duration of the operation (cache retrieval).
                    op_duration_ms = (time.perf_counter() - op_start_time) * 1000
                    # Log a message indicating a cache hit and the operation duration.
                    self.logger.info(f"Request {tts_request.request_id}: Cache hit. Duration: {op_duration_ms:.2f}ms.")
                    # Return the cached audio content and metadata.
                    return cached_audio, metadata
            # If caching is disabled or if it's a cache miss:
            # Update cache metrics to record a cache miss.
            self.cost_metrics.update_cache_metrics(cache_hit=False)
            # Set the 'source' in metadata to "api_synthesized".
            metadata["source"] = "api_synthesized"

            # --- Step 5: Calculate Estimated Cost ---
            # Calculate the estimated cost for the synthesis, now that the actual voice/quality is known.
            tts_request.estimated_cost = self._calculate_estimated_cost(tts_request, actual_quality_tier)
            # Add the estimated cost to the 'metadata'.
            metadata["estimated_cost_usd"] = tts_request.estimated_cost

            # --- Step 6: Prepare for API Call (acquire semaphore, rate limit) ---
            # Asynchronously acquire the semaphore to limit concurrency for I/O-bound API calls.
            async with self._async_semaphore:
                # Wait according to the adaptive rate limiter before making the API call.
                await self.rate_limiter.acquire()

                # Construct the SynthesisInput arguments based on whether SSML is enabled.
                synthesis_input_args = {}
                # Check if SSML is enabled for the request.
                if tts_request.ssml_enabled:
                    # If SSML is enabled, set the 'ssml' field in arguments.
                    synthesis_input_args["ssml"] = tts_request.text
                # Handle plain text input.
                else:
                    # If SSML is not enabled, set the 'text' field in arguments.
                    synthesis_input_args["text"] = tts_request.text
                # Create the SynthesisInput object using the prepared arguments.
                synthesis_input = texttospeech.SynthesisInput(**synthesis_input_args)

                # Construct the VoiceSelectionParams object.
                voice_params = texttospeech.VoiceSelectionParams(
                    language_code=tts_request.language_code,
                    name=final_voice_name,
                    ssml_gender=ssml_gender
                )

                # Construct the AudioConfig object.
                audio_config = texttospeech.AudioConfig(
                    audio_encoding=tts_request.audio_encoding,
                    speaking_rate=tts_request.speaking_rate,
                    pitch=tts_request.pitch,
                )

                # Define the actual API call function using functools.partial to pre-fill arguments.
                # This creates a callable for self.tts_client.synthesize_speech with fixed input, voice, and audio_config.
                api_call_func = functools.partial(
                    self.tts_client.synthesize_speech, # type: ignore # google-cloud-texttospeech client methods are not perfectly typed for partial
                    input=synthesis_input,
                    voice=voice_params,
                    audio_config=audio_config
                )

                # --- Step 7: Execute API Call (with Circuit Breaker and client-side retry) ---
                # Log a debug message indicating that an API call is about to be made.
                self.logger.debug(f"Request {tts_request.request_id}: Making API call to tts_client.synthesize_speech.")
                # Initialize 'response' to None.
                response: Optional[texttospeech.SynthesizeSpeechResponse] = None

                # Check if the circuit breaker is enabled.
                if self.circuit_breaker:
                    # Execute the API call via the circuit breaker, running the (potentially blocking) Google client call in a separate thread.
                    response = await self.circuit_breaker.execute_async(
                        # Lambda function to execute the 'api_call_func' in a thread pool executor.
                        lambda: asyncio.to_thread(api_call_func)
                    )
                # Handle direct execution if the circuit breaker is not enabled.
                else:
                    # Execute the (potentially blocking) Google client call directly in a separate thread.
                    response = await asyncio.to_thread(api_call_func)

                # Check if the API response is valid and contains audio content.
                if not response or not response.audio_content:
                    # If the response is invalid or lacks audio content, raise a SynthesisFailedError.
                    raise SynthesisFailedError(f"Request {tts_request.request_id}: API call returned empty response or no audio content.")

                # If the API call was successful, record success with the rate limiter.
                await self.rate_limiter.record_success()
                # Extract the audio content from the API response.
                audio_content = response.audio_content
                # Get the billable character count for the request and add it to metadata.
                metadata["character_count"] = tts_request.get_billable_character_count()

                # --- Step 8: Update Cost Metrics (after successful synthesis) ---
                # Calculate the duration of the synthesis operation.
                op_duration_ms = (time.perf_counter() - op_start_time) * 1000
                # Update cost metrics with details of the successful synthesis.
                self.cost_metrics.update_on_synthesis_success(
                    character_count=metadata["character_count"],
                    quality_tier=actual_quality_tier,
                    cost=tts_request.estimated_cost,
                    response_time_ms=op_duration_ms
                )

                # --- Step 9: Cache the Result (if caching enabled) ---
                # Check if caching is enabled for the orchestrator.
                if self.enable_caching:
                    # Put the synthesized audio content into the cache.
                    await self._put_in_cache(tts_request.cache_key, audio_content)

                # Log a message indicating successful synthesis and relevant details.
                self.logger.info(f"Request {tts_request.request_id}: Synthesis successful. Duration: {op_duration_ms:.2f}ms. "
                                 f"Chars: {metadata['character_count']}, Cost: ${tts_request.estimated_cost:.6f}.")
                # Return the synthesized audio content and metadata.
                return audio_content, metadata

        # Catch ValidationError if input validation for TTSRequest fails.
        except ValidationError as e:
            # Log the validation error.
            self.logger.error(f"Request {tts_request.request_id}: Validation error - {e}", exc_info=True)
            # Set the error message in metadata.
            metadata["error"] = f"Validation Error: {e}"
            # Record a synthesis failure in cost metrics.
            self.cost_metrics.record_synthesis_failure()
            # Return None for audio content and the updated metadata.
            return None, metadata
        # Catch VoiceSelectionError if intelligent voice selection fails.
        except VoiceSelectionError as e:
            # Log the voice selection error.
            self.logger.error(f"Request {tts_request.request_id}: Voice selection error - {e}", exc_info=True)
            # Set the error message in metadata.
            metadata["error"] = f"Voice Selection Error: {e}"
            # Record a synthesis failure in cost metrics.
            self.cost_metrics.record_synthesis_failure()
            # Return None for audio content and the updated metadata.
            return None, metadata
        # Catch APICallError, which can be raised by the circuit breaker or direct API call issues.
        except APICallError as e:
            # Log the API call error.
            self.logger.error(f"Request {tts_request.request_id}: API call error during synthesis - {e}", exc_info=True)
            # Set the error message in metadata.
            metadata["error"] = f"API Call Error: {e}"
            # Check if the underlying cause of the APICallError is a Google API ResourceExhausted error (rate limiting).
            is_rate_limit_err = isinstance(e.__cause__, gapi_exceptions.ResourceExhausted)
            # Record failure with the rate limiter, indicating if it was a rate limit error.
            await self.rate_limiter.record_failure(is_rate_limit_error=is_rate_limit_err)
            # Record a synthesis failure in cost metrics.
            self.cost_metrics.record_synthesis_failure()
            # Return None for audio content and the updated metadata.
            return None, metadata
        # Catch GoogleCloudError for specific errors from Google Cloud client libraries.
        except GoogleCloudError as e:
            # Log the Google Cloud error.
            self.logger.error(f"Request {tts_request.request_id}: Google Cloud error during synthesis - {e}", exc_info=True)
            # Set the error message in metadata.
            metadata["error"] = f"Google Cloud Error: {e}"
            # Check if the GoogleCloudError is a ResourceExhausted error (rate limiting).
            is_rate_limit_err = isinstance(e, gapi_exceptions.ResourceExhausted)
            # Record failure with the rate limiter, indicating if it was a rate limit error.
            await self.rate_limiter.record_failure(is_rate_limit_error=is_rate_limit_err)
            # Record a synthesis failure in cost metrics.
            self.cost_metrics.record_synthesis_failure()
            # Return None for audio content and the updated metadata.
            return None, metadata
        # Catch any other unexpected exceptions.
        except Exception as e:
            # Log the unexpected critical error.
            self.logger.critical(f"Request {tts_request.request_id}: Unexpected critical error during synthesis - {e}", exc_info=True)
            # Set the error message in metadata.
            metadata["error"] = f"Unexpected Critical Error: {e}"
            # Record failure with the rate limiter, assuming a general failure.
            await self.rate_limiter.record_failure()
            # Record a synthesis failure in cost metrics.
            self.cost_metrics.record_synthesis_failure()
            # Return None for audio content and the updated metadata.
            return None, metadata

    # Defines a method to retrieve a comprehensive analysis of current TTS usage costs and recommendations.
    def get_cost_analysis(self) -> Dict[str, Any]:
        """
        Returns a comprehensive analysis of current TTS usage costs, free tier status,
        and actionable optimization recommendations.

        Returns:
            Dict[str, Any]: A dictionary containing detailed cost and usage metrics,
                            remaining free tier allocations, and optimization advice.
        """
        # Log an informational message indicating the start of cost analysis report generation.
        self.logger.info("Generating cost analysis report...")
        # Get a thread-safe snapshot of the current cost metrics.
        current_metrics = self.cost_metrics.get_current_metrics_snapshot()

        # Get the free tier character limit for STANDARD voice quality.
        standard_free_limit = VoiceQuality.STANDARD.get_free_tier_character_limit()
        # Get the free tier character limit for premium voice quality (using NEURAL2 as representative).
        premium_free_limit = VoiceQuality.NEURAL2.get_free_tier_character_limit()

        # Calculate the remaining free tier characters for standard voices.
        remaining_standard_free_chars = max(0, standard_free_limit - current_metrics['standard_voice_usage'])
        # Calculate the remaining free tier characters for premium voices.
        remaining_premium_free_chars = max(0, premium_free_limit - current_metrics['premium_voice_usage'])

        # Generate optimization recommendations based on the current metrics snapshot.
        recommendations = self._generate_optimization_recommendations(current_metrics)

        # Construct the comprehensive cost analysis report dictionary.
        analysis_report = {
            "report_generated_utc": datetime.now(timezone.utc).isoformat(),
            "metrics_last_reset_utc": current_metrics['last_reset_utc'],
            "current_usage_metrics": {
                "total_characters_processed": current_metrics['characters_processed'],
                "standard_voice_characters": current_metrics['standard_voice_usage'],
                "premium_voice_characters": current_metrics['premium_voice_usage'],
                "successful_requests": current_metrics['successful_requests'],
                "failed_requests": current_metrics['failed_requests'],
            },
            "free_tier_status": {
                "standard_voice_free_tier_limit": standard_free_limit,
                "standard_voice_remaining_free_chars": remaining_standard_free_chars,
                "premium_voice_free_tier_limit": premium_free_limit,
                "premium_voice_remaining_free_chars": remaining_premium_free_chars,
            },
            "cost_information": {
                "estimated_monthly_cost_usd": current_metrics['estimated_monthly_cost'],
                "defined_cost_threshold_usd": self.cost_threshold_usd,
            },
            "performance_metrics": {
                "cache_hit_rate": f"{current_metrics['cache_hit_rate']:.2%}",
                "cache_hits": current_metrics['cache_hits'],
                "cache_misses": current_metrics['cache_misses'],
                "avg_api_response_time_ms": f"{current_metrics['avg_response_time_ms']:.2f}",
            },
            "optimization_recommendations": recommendations
        }
        # Log an informational message indicating successful generation of the cost analysis report.
        self.logger.info("Cost analysis report generated successfully.")
        # Return the constructed 'analysis_report'.
        return analysis_report

    # Defines an internal helper method to generate cost optimization recommendations based on current metrics.
    def _generate_optimization_recommendations(self, current_metrics: Dict[str, Any]) -> List[str]:
        """
        Generates a list of cost optimization recommendations based on current usage metrics.
        This is a helper method for `get_cost_analysis`.

        Args:
            current_metrics: A snapshot dictionary of current cost metrics.

        Returns:
            List[str]: A list of string recommendations.
        """
        # Log a debug message indicating the start of optimization recommendation generation.
        self.logger.debug("Generating optimization recommendations based on current metrics...")
        # Initialize an empty list to store the generated recommendations.
        recommendations: List[str] = []
        # Define a warning factor for approaching the cost threshold (e.g., 90%).
        cost_threshold_factor_warn = 0.9
        # Define a warning factor for approaching free tier usage limits (e.g., 80%).
        free_tier_usage_factor_warn = 0.8

        # Get the free tier character limit for premium voice quality.
        premium_free_limit = VoiceQuality.NEURAL2.get_free_tier_character_limit()
        # Check if premium voice usage is approaching or has exceeded its free tier limit.
        if current_metrics['premium_voice_usage'] > premium_free_limit * free_tier_usage_factor_warn:
            # Construct a recommendation message for premium voice usage.
            recommendation_msg = (
                f"Premium voice usage ({current_metrics['premium_voice_usage']:,} chars) is "
                f"approaching/exceeded its monthly free tier limit ({premium_free_limit:,} chars). "
                "Consider shifting non-critical requests to Standard voices to optimize costs."
            )
            # Add the recommendation message to the list.
            recommendations.append(recommendation_msg)

        # Get the free tier character limit for standard voice quality.
        standard_free_limit = VoiceQuality.STANDARD.get_free_tier_character_limit()
        # Check if standard voice usage is approaching the end of its free tier.
        if standard_free_limit * free_tier_usage_factor_warn < current_metrics['standard_voice_usage'] <= standard_free_limit :
             # Construct a recommendation message for standard voice usage.
            recommendation_msg = (
                f"Standard voice usage ({current_metrics['standard_voice_usage']:,} chars) is "
                f"approaching the end of its free tier ({standard_free_limit:,} chars). Monitor closely to anticipate costs."
            )
             # Add the recommendation message to the list.
            recommendations.append(recommendation_msg)

        # Check if the estimated monthly cost is approaching or has exceeded the defined cost threshold.
        if current_metrics['estimated_monthly_cost'] > self.cost_threshold_usd * cost_threshold_factor_warn:
            # Construct a recommendation message regarding the cost threshold.
            recommendation_msg = (
                f"Estimated monthly cost (${current_metrics['estimated_monthly_cost']:.2f}) is "
                f"approaching/exceeded the defined threshold (${self.cost_threshold_usd:.2f}). "
                "Review usage patterns and consider stricter caching, batching, or quality tier adjustments."
            )
            # Add the recommendation message to the list.
            recommendations.append(recommendation_msg)

        # Calculate the total number of cache lookups.
        total_cache_lookups = current_metrics['cache_hits'] + current_metrics['cache_misses']
        # Check if caching is enabled and if there's enough data for a meaningful cache hit rate analysis.
        if self.enable_caching and total_cache_lookups > 20:
            # Check if the cache hit rate is below a certain threshold (e.g., 20%).
            if current_metrics['cache_hit_rate'] < 0.20:
                # Construct a recommendation message for low cache hit rate.
                recommendation_msg = (
                    f"Cache hit rate is low ({current_metrics['cache_hit_rate']:.2%}). "
                    "Review request patterns for frequently repeated synthesis tasks. "
                    "Ensure cache TTL is appropriate for your use case."
                )
                # Add the recommendation message to the list.
                recommendations.append(recommendation_msg)
        # Else, if caching is disabled but there has been significant successful request activity.
        elif not self.enable_caching and current_metrics['successful_requests'] > 50 :
             # Construct a recommendation message suggesting enabling caching.
             recommendation_msg = (
                "Caching is currently disabled. Enabling caching could significantly reduce costs "
                "and improve response times for repeated synthesis requests."
            )
             # Add the recommendation message to the list.
            recommendations.append(recommendation_msg)

        # Calculate the total number of API attempts (successful + failed requests).
        total_api_attempts = current_metrics['successful_requests'] + current_metrics['failed_requests']
        # Check if there's enough data and if there have been any failed requests.
        if total_api_attempts > 20 and current_metrics['failed_requests'] > 0:
            # Calculate the API request failure rate.
            failure_rate = current_metrics['failed_requests'] / total_api_attempts
            # Check if the failure rate is above a certain threshold (e.g., 10%).
            if failure_rate > 0.10:
                # Construct a recommendation message for high API failure rate.
                recommendation_msg = (
                    f"High API request failure rate detected ({failure_rate:.2%}). "
                    "Investigate logs for common errors (e.g., invalid input, API issues, network problems, quota limits)."
                )
                # Add the recommendation message to the list.
                recommendations.append(recommendation_msg)

        # Define a threshold for considering average API response time as high (e.g., 1500 ms).
        avg_response_time_threshold_ms = 1500
        # Check if average response time is high and there's enough data from successful requests.
        if current_metrics['avg_response_time_ms'] > avg_response_time_threshold_ms and current_metrics['successful_requests'] > 10 :
            # Construct a recommendation message for high average API response time.
            recommendation_msg = (
                f"Average API response time is high ({current_metrics['avg_response_time_ms']:.2f}ms). "
                "Consider optimizing request text length, checking network latency, or ensuring "
                "sufficient concurrency settings if using batch processing."
            )
            # Add the recommendation message to the list.
            recommendations.append(recommendation_msg)

        # Check if no specific recommendations were generated.
        if not recommendations:
            # Add a default message indicating that usage appears to be within parameters.
            recommendations.append("Current TTS usage appears to be within defined operational parameters. No immediate optimization actions flagged.")

        # Log a debug message indicating the number of generated optimization recommendations.
        self.logger.debug(f"Generated {len(recommendations)} optimization recommendations.")
        # Return the list of recommendation strings.
        return recommendations

    # Defines an asynchronous method to process a batch of TTSRequest objects concurrently.
    async def batch_synthesize(self, requests_list: List[TTSRequest]) -> List[Tuple[Optional[bytes], Dict[str, Any]]]:
        """
        Processes a batch of TTSRequest objects concurrently with cost optimization,
        adaptive rate limiting, and circuit breaker patterns applied to each individual request.

        Args:
            requests_list: A list of TTSRequest objects to be synthesized.

        Returns:
            A list of tuples, where each tuple corresponds to an input TTSRequest and contains:
            - Optional[bytes]: Synthesized audio content (bytes) if successful, else None.
            - Dict[str, Any]: Metadata about the synthesis operation for that request.
            The order of results in the list matches the order of requests in the input list.
        """
        # Check if the input 'requests_list' is empty.
        if not requests_list:
            # Log an informational message if an empty list is provided.
            self.logger.info("Batch synthesize called with an empty list of requests.")
            # Return an empty list as there are no requests to process.
            return []

        # Log an informational message indicating the start of batch synthesis and the number of requests.
        self.logger.info(f"Starting batch synthesis for {len(requests_list)} requests.")

        # Define a helper function 'sort_key' to determine the sorting order for requests.
        # Requests are sorted by priority (ascending), then by preliminary estimated cost (ascending).
        def sort_key(req: TTSRequest):
            # Get the preferred quality tier from the request for preliminary cost estimation.
            prelim_quality = req.quality_tier
            # Get the billable character count from the request.
            prelim_char_count = req.get_billable_character_count()
            # Calculate a preliminary estimated cost for sorting purposes.
            prelim_cost = (prelim_char_count * prelim_quality.get_cost_per_million_characters()) / 1_000_000.0
            # Return a tuple of (priority, preliminary_cost) for sorting.
            return (req.priority, prelim_cost)

        # Initialize 'sorted_requests' with the original 'requests_list'.
        sorted_requests: List[TTSRequest]
        # Begin a try block to handle potential exceptions during request sorting.
        try:
            # Sort the 'requests_list' using the defined 'sort_key'.
            sorted_requests = sorted(requests_list, key=sort_key)
        # Catch any exception that occurs during sorting.
        except Exception as e:
            # Log an error message if sorting fails, and process requests in their original order.
            self.logger.error(f"Error during sorting of batch requests: {e}. Processing in original order.", exc_info=True)
            # Assign the original 'requests_list' to 'sorted_requests' as a fallback.
            sorted_requests = requests_list

        # Create a list of asyncio tasks, where each task is a call to 'self.synthesize_speech' for a request.
        tasks = [self.synthesize_speech(req) for req in sorted_requests]

        # Execute all tasks concurrently using asyncio.gather.
        # 'return_exceptions=True' ensures that exceptions in tasks are returned as results rather than stopping 'gather'.
        self.logger.info(f"Executing {len(tasks)} synthesis tasks concurrently for batch.")
        # Await the completion of all tasks and get their results (or exceptions).
        results_from_gather = await asyncio.gather(*tasks, return_exceptions=True)

        # Initialize an empty list to store the final results in the processed order.
        final_results: List[Tuple[Optional[bytes], Dict[str, Any]]] = []
        # Iterate through the results obtained from 'asyncio.gather', along with their original index.
        for i, result_item in enumerate(results_from_gather):
            # Get the request ID from the corresponding request in the 'sorted_requests' list for correlation.
            original_request_id = sorted_requests[i].request_id
            # Check if the 'result_item' is an instance of Exception (meaning the task failed).
            if isinstance(result_item, Exception):
                # Log an error message for the failed batch request, including the exception.
                self.logger.error(f"Batch request {original_request_id} (index {i}) failed with exception: {result_item}", exc_info=result_item)
                # Create an error metadata dictionary for the failed request.
                error_metadata = {
                    "request_id": original_request_id,
                    "error": f"Unhandled Batch Exception: {type(result_item).__name__} - {str(result_item)}",
                    "source": "batch_processor_error",
                    "timestamp_utc": datetime.now(timezone.utc).isoformat()
                }
                # Append a tuple (None for audio, error_metadata) to 'final_results'.
                final_results.append((None, error_metadata))
            # Check if the 'result_item' is a tuple of length 2 (expected format for successful/handled synthesis).
            elif isinstance(result_item, tuple) and len(result_item) == 2:
                # Unpack the tuple into 'audio_data' and 'metadata'.
                audio_data, metadata = result_item # type: ignore # mypy struggles with heterogeneous list from gather
                # Ensure 'metadata' always has 'request_id' for correlation, using 'original_request_id' as fallback.
                metadata["request_id"] = metadata.get("request_id", original_request_id)
                # Append the 'audio_data' and 'metadata' to 'final_results'.
                final_results.append((audio_data, metadata))
            # Handle cases where the result format from 'asyncio.gather' is unexpected.
            else:
                 # Log an error message for the unexpected result format.
                 self.logger.error(f"Batch request {original_request_id} (index {i}) returned unexpected result format: {result_item}")
                 # Create an error metadata dictionary for the unexpected result.
                 unexpected_error_metadata = {
                    "request_id": original_request_id,
                    "error": f"Unexpected result format from synthesis task: {type(result_item).__name__}",
                    "source": "batch_processor_unexpected_result",
                    "timestamp_utc": datetime.now(timezone.utc).isoformat()
                }
                 # Append a tuple (None for audio, unexpected_error_metadata) to 'final_results'.
                 final_results.append((None, unexpected_error_metadata))

        # Log an informational message indicating the completion of batch synthesis.
        self.logger.info(f"Batch synthesis completed for {len(requests_list)} requests.")
        # Return the 'final_results' list, which contains results in the order of 'sorted_requests'.
        return final_results


In [ ]:
# Defines an asynchronous function to demonstrate production-like usage of the VertexTTSOrchestrator.

async def main_usage_example():
    """
    Demonstrates production-like usage of the VertexTTSOrchestrator,
    showcasing single synthesis, batch synthesis, error handling, and cost analysis.
    Configuration is partially driven by environment variables.
    """
    # Obtain a logger instance specific to this "MainUsageExample".
    main_logger = logging.getLogger("MainUsageExample")
    # Log an informational message indicating the start of the usage example.
    main_logger.info("Starting VertexTTSOrchestrator usage example...")

    # Retrieve the Google Cloud Project ID from the "GOOGLE_CLOUD_PROJECT" environment variable.
    project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
    # Check if the 'project_id' was successfully retrieved.
    if not project_id:
        # Log an error message if the environment variable is not set.
        main_logger.error("GOOGLE_CLOUD_PROJECT environment variable not set. Cannot proceed.")
        # Print an error message to the console.
        print("Error: GOOGLE_CLOUD_PROJECT environment variable is required.")
        # Return from the function as the required configuration is missing.
        return

    # Retrieve the Google Cloud Region from the "GOOGLE_CLOUD_REGION" environment variable,
    # defaulting to 'VertexTTSOrchestrator.DEFAULT_REGION' if not set.
    region = os.getenv("GOOGLE_CLOUD_REGION", VertexTTSOrchestrator.DEFAULT_REGION)
    # Retrieve the path to Google Application Credentials from the "GOOGLE_APPLICATION_CREDENTIALS" environment variable.
    credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") # This will be None if not set.

    # Log the Project ID and Region being used.
    main_logger.info(f"Using Project ID: {project_id}, Region: {region}")
    # Check if a 'credentials_path' was provided.
    if credentials_path:
        # Log the path to the credentials file if it's being used.
        main_logger.info(f"Using Credentials Path: {credentials_path}")
    # Handle the case where no specific credentials path is provided.
    else:
        # Log that Application Default Credentials will be used.
        main_logger.info("Using Application Default Credentials.")

    # Initialize the 'orchestrator' variable to None. It will hold the VertexTTSOrchestrator instance.
    orchestrator: Optional[VertexTTSOrchestrator] = None
    # Begin a try block to handle potential errors during orchestrator initialization.
    try:
        # Instantiate the VertexTTSOrchestrator with specified and default configurations.
        orchestrator = VertexTTSOrchestrator(
            project_id=project_id,
            region=region,
            enable_caching=True,
            cost_threshold_usd=50.0,
            max_concurrent_requests=5,
            google_application_credentials_path=credentials_path
        )
    # Catch ConfigurationError or InitializationError specifically if they occur during instantiation.
    except (ConfigurationError, InitializationError) as e:
        # Log a critical error message if orchestrator initialization fails.
        main_logger.critical(f"Failed to initialize TTS Orchestrator: {e}", exc_info=True)
        # Print a critical error message to the console.
        print(f"Critical Error: Could not initialize TTS Orchestrator. {e}")
        # Return from the function as the orchestrator could not be initialized.
        return
    # Catch any other unexpected Exception during orchestrator initialization.
    except Exception as e:
        # Log a critical error message for any unexpected initialization error.
        main_logger.critical(f"Unexpected critical error during orchestrator initialization: {e}", exc_info=True)
        # Print an unexpected critical error message to the console.
        print(f"Unexpected Critical Error during initialization. {e}")
        # Return from the function.
        return

    # --- Example 1: Single TTS Requests ---
    # Log a section header for demonstrating single TTS requests.
    main_logger.info("\n--- Demonstrating Single TTS Requests ---")
    # Define a list of dictionaries, each representing parameters for a single TTS request.
    single_requests_data = [
        {"text": "Hello from the Text-to-Speech Orchestrator! This is a standard voice.", "lang": "en-US", "quality": VoiceQuality.STANDARD},
        {"text": "This is a premium quality Neural2 voice, for critical applications.", "lang": "en-US", "quality": VoiceQuality.NEURAL2},
        {"text": "Bonjour le monde, ceci est une voix française.", "lang": "fr-FR", "quality": VoiceQuality.STANDARD},
        {"text": "<speak>This is <emphasis level='strong'>SSML</emphasis> text with a <break time='500ms'/> pause.</speak>", "lang": "en-GB", "quality": VoiceQuality.NEURAL2, "ssml": True},
        {"text": "A short request for caching.", "lang": "en-US", "quality": VoiceQuality.STANDARD},
        {"text": "A short request for caching.", "lang": "en-US", "quality": VoiceQuality.STANDARD}, # Expected cache hit
        {"text": "これは日本語の音声合成のサンプルです。", "lang": "ja-JP", "quality": VoiceQuality.NEURAL2},
        {"text": "Invalid request with very long text" * 1000, "lang": "en-US", "quality": VoiceQuality.STANDARD}, # Expected to fail TTSRequest validation
    ]

    # Iterate through the 'single_requests_data' list with an index 'i' and request data 'req_data'.
    for i, req_data in enumerate(single_requests_data):
        # Construct an output filename for the synthesized audio.
        output_filename = f"output_single_{i}_{req_data['lang']}.mp3"
        # Begin a try block to handle potential errors during the processing of a single request.
        try:
            # Attempt to create a TTSRequest object using data from the current 'req_data' dictionary.
            tts_req = TTSRequest(
                text=req_data["text"],
                language_code=req_data["lang"],
                quality_tier=req_data["quality"],
                ssml_enabled=req_data.get("ssml", False)
            )
            # Log an informational message about processing the current single request.
            main_logger.info(f"Processing single request {tts_req.request_id} ({output_filename})...")
            # Call the orchestrator's synthesize_speech method with the created TTSRequest object.
            # This returns a tuple: (audio_data_bytes_or_None, metadata_dictionary).
            audio_data, metadata = await orchestrator.synthesize_speech(tts_req)

            # Check if 'audio_data' was successfully returned (i.e., not None).
            if audio_data:
                # Open the specified 'output_filename' in binary write mode ('wb').
                with open(output_filename, "wb") as f:
                    # Write the 'audio_data' bytes to the opened file.
                    f.write(audio_data)
                # Log a success message including the request ID, output filename, and metadata.
                main_logger.info(f"SUCCESS: Request {tts_req.request_id} -> Saved to {output_filename}. Metadata: {metadata}")
            # Handle cases where 'audio_data' is None, indicating a failure in synthesis.
            else:
                # Log an error message for the failed request, including request ID and metadata.
                main_logger.error(f"FAILURE: Request {tts_req.request_id} -> No audio data returned. Metadata: {metadata}")

        # Catch ValidationError if TTSRequest creation fails due to invalid input data.
        except ValidationError as e:
            # Log an error message indicating that the request data failed validation and was skipped.
            main_logger.error(f"SKIPPED: Request data for '{output_filename}' failed validation: {e}")
        # Catch any other unexpected Exception during the processing of a single request.
        except Exception as e:
            # Log a critical error message for unexpected errors during single request processing.
            main_logger.critical(f"CRITICAL ERROR during single request for '{output_filename}': {e}", exc_info=True)

    # --- Example 2: Batch TTS Requests ---
    # Log a section header for demonstrating batch TTS requests.
    main_logger.info("\n--- Demonstrating Batch TTS Requests ---")
    # Define a list of TTSRequest objects for batch processing.
    batch_requests_data: List[TTSRequest] = [
        TTSRequest(text="Batch item one, standard quality.", language_code="en-US", quality_tier=VoiceQuality.STANDARD, priority=1),
        TTSRequest(text="Batch item two, premium quality.", language_code="en-GB", quality_tier=VoiceQuality.NEURAL2, priority=5),
        TTSRequest(text="Batch item three, another standard.", language_code="es-ES", quality_tier=VoiceQuality.STANDARD, priority=2),
        TTSRequest(text="<speak>Batch SSML <prosody rate='slow'>slowly spoken</prosody>.</speak>", language_code="en-US", quality_tier=VoiceQuality.NEURAL2, ssml_enabled=True, priority=3),
        TTSRequest(text="Batch item five, likely to be cached.", language_code="en-US", quality_tier=VoiceQuality.STANDARD, priority=1),
    ]
    # Begin a try block to handle potential errors during batch synthesis.
    try:
        # Call the orchestrator's batch_synthesize method with the list of TTSRequest objects.
        # This returns a list of tuples, each containing (audio_data_or_None, metadata_dictionary).
        batch_results = await orchestrator.batch_synthesize(batch_requests_data)
        # Iterate through the 'batch_results' list with an index 'i' and result tuple '(audio_data, metadata)'.
        for i, (audio_data, metadata) in enumerate(batch_results):
            # Get the request ID from metadata, defaulting if not present.
            req_id = metadata.get("request_id", f"batch_unknown_{i}")
            # Construct an output filename for the synthesized audio from the batch request.
            output_filename = f"output_batch_{i}_{req_id}.mp3"
            # Check if 'audio_data' was successfully returned for this batch item.
            if audio_data:
                # Open the specified 'output_filename' in binary write mode ('wb').
                with open(output_filename, "wb") as f:
                    # Write the 'audio_data' bytes to the opened file.
                    f.write(audio_data)
                # Log a success message for the batch request item.
                main_logger.info(f"SUCCESS: Batch request {req_id} -> Saved to {output_filename}. Metadata: {metadata}")
            # Handle cases where 'audio_data' is None for a batch item.
            else:
                # Log an error message for the failed batch request item.
                main_logger.error(f"FAILURE: Batch request {req_id} -> Synthesis failed. Metadata: {metadata}")
    # Catch any unexpected Exception during the batch synthesis process.
    except Exception as e:
        # Log a critical error message for unexpected errors during batch synthesis.
        main_logger.critical(f"CRITICAL ERROR during batch synthesis: {e}", exc_info=True)

    # --- Example 3: Get Cost Analysis ---
    # Log a section header for demonstrating cost analysis retrieval.
    main_logger.info("\n--- Generating Cost Analysis Report ---")
    # Begin a try block to handle potential errors during cost analysis generation.
    try:
        # Call the orchestrator's get_cost_analysis method to retrieve the cost report.
        cost_analysis = orchestrator.get_cost_analysis()
        # Log the cost analysis report as a pretty-printed JSON string.
        main_logger.info("Cost Analysis Report:\n" + json.dumps(cost_analysis, indent=2))
        # Print a header for the cost analysis report to the console.
        print("\n=== Cost Analysis Report ===")
        # Print the cost analysis report as a pretty-printed JSON string to the console.
        print(json.dumps(cost_analysis, indent=2))
        # Print a footer for the cost analysis report to the console.
        print("==========================")
    # Catch any unexpected Exception during cost analysis generation.
    except Exception as e:
        # Log an error message if cost analysis generation fails.
        main_logger.error(f"Error generating cost analysis: {e}", exc_info=True)

    # Log an informational message indicating the end of the usage example.
    main_logger.info("VertexTTSOrchestrator usage example finished.")

# Check if the script is being run as the main module.
if __name__ == "__main__":
    # Begin a try block to handle top-level exceptions during script execution.
    try:
        # Run the 'main_usage_example' asynchronous function using asyncio.run().
        asyncio.run(main_usage_example())
    # Catch KeyboardInterrupt (e.g., Ctrl+C) to allow graceful exit.
    except KeyboardInterrupt:
        # Log an informational message if the script is interrupted by the user.
        logger.info("Main example interrupted by user (KeyboardInterrupt). Exiting.")
    # Catch any other unexpected Exception at the top level of script execution.
    except Exception as e:
        # Log a critical error message for unhandled exceptions in the main execution block.
        logger.critical(f"Critical unhandled exception in main: {e}", exc_info=True)
